In [1]:
# !pip install pickle5  # pickle is part of the standard library, but pickle5 can be used for enhanced features
# !pip install numpy
# !pip install requests
# !pip install pandas
# !pip install glob2  # glob is part of the standard library, but glob2 is an enhanced version
# !pip install scipy
# !pip install matplotlib
# !pip install seaborn
# !pip install tabulate

import pickle
import numpy as np
from multiprocessing import Pool

import requests, json
import pandas as pd
import glob
import time
from multiprocessing import Pool
import numpy as np
from scipy import stats

def toc(start_time):
    elapsed = timeit.default_timer() - start_time
    print(elapsed)
    
# from zipfile import ZipFile
import re
# from pprint import pprint
# import glob
import pickle

import matplotlib as mpl
import seaborn as sns
import matplotlib.cm as cm
import pickle
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import gridspec
import matplotlib.transforms as mtransforms
sns.set_theme(style='white')

%matplotlib inline
%config InlineBackend.figure_format = 'svg'

plt.rcParams['xtick.major.size'] = 3
plt.rcParams['xtick.major.width'] = 1
plt.rcParams['xtick.bottom'] = True
plt.rcParams['ytick.left'] = True
sns.set_context('talk', font_scale=.65)
mpl.rcParams.update({'text.usetex': False})

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import pandas as pd
# # Choose an embedding backend
# !pip install bertopic[flair, gensim, spacy, use]

# # Topic modeling with images
# !pip install bertopic[vision]
# !pip install xgi==0.7.3
# !pip install distinctipy
# Load document info and topic frequencies

# --- Define sector keywords as before ---
sector_keyword_strength = {
    'transport_strong': [
        "electric vehicle", "evs", "bev", "battery electric", "battery-electric vehicle",
        "tesla model", "model 3", "model y", "chevy bolt", "nissan leaf",
        "ioniq 5", "mustang mach-e", "id.4", "rivian", "lucid air",
        "supercharger", "gigafactory", "zero emission vehicle", "zero-emission vehicle",
        "pure electric", "all-electric", "fully electric", "100% electric",
        "electric powertrain", "electric drivetrain", "electric motor vehicle",
        "level 2 charger", "dc fast charger", "public charger", "home charger",
        "charging network", "range anxiety", "mpge",
        "bike lane", "protected cycleway", "car-free", "low emission zone"
    ],
    'transport_weak': [
        "electric car", "electric truck", "electric suv", "plug-in hybrid",
        "phev", "charging station", "charge point", "kw charger", 
        "battery swap", "solid-state battery", "gigacast",
        "tax credit", "zev mandate", "ev rebate", "phase-out ice",
        "e-bike", "micro-mobility", "last-mile delivery", "transit electrification",
        "tesla", "spacex launch price?", "elon says",
        "rail electrification", "hydrogen truck", "low carbon transport"
    ],
    'housing_strong': [
        "rooftop solar", "solar pv", "pv panel", "photovoltaics",
        "solar array", "net metering", "feed-in tariff", "solar inverter",
        "kwh generated", "solar roof", "sunrun", "sunpower",
        r"solar\s+panel(s)?", r"solar\s+pv", r"rooftop\s+solar",
        r"solar\s+power", r"photovoltaic(s)?"
    ],
    'housing_weak': [
        "solar panels", "solar power", "solar installer",
        "battery storage", "powerwall", "home battery", "smart thermostat",
        "energy audit", "energy efficiency upgrade", "led retrofit",
        "green home", "net-zero house", "zero-energy building",
        "solar tax credit", "pvgis", "renewable portfolio standard",
        "community solar", "virtual power plant", "rooftop rebate"
    ],
    'food_strong': [
        "vegan", "plant-based diet", "veganism", "veganuary", "vegetarian", "veg lifestyle",
        "carnivore diet", "meat lover", "steakhouse", "barbecue festival",
        "bacon double", "grass-fed beef", "factory farming",
        "meatless monday", "beyond meat", "impossible burger",
        "plant-based burger", "animal cruelty free"
    ],
    'food_weak': [
        "red meat", "beef consumption", "dairy free", "plant protein",
        "soy burger", "nutritional yeast", "seitan", "tofurky",
        "agricultural emissions", "methane footprint", "carbon hoofprint",
        "cow burps", "livestock emissions", "feedlot",
        "recipe vegan", "tofu scramble", "almond milk", "oat milk",
        "flexitarian", "climatetarian",
        "cultivated meat", "lab-grown meat", "precision fermentation"
    ]
}

# Combine all keywords for regex search
sector_keywords = (
    sector_keyword_strength['transport_strong'] +
    sector_keyword_strength['transport_weak'] +
    sector_keyword_strength['housing_strong'] +
    sector_keyword_strength['housing_weak'] +
    sector_keyword_strength['food_strong'] +
    sector_keyword_strength['food_weak']
)

## climate subreddit only

In [4]:

# import os
# import pandas as pd

# # Directory containing the CSVs
# csv_dir = r"C:\Users\chowdhary\Desktop\Redit project 4.09.2024\data\reddit_subreddits24_csv"

# # Get all CSV files ending with ".csv" (case insensitive), except those with "stupid" in the name
# all_csvs = [
#     os.path.join(csv_dir, f)
#     for f in os.listdir(csv_dir)
#     if f.lower().endswith(".csv")
#     and "stupid" not in f.lower()
# ]

# # Separate submission and comment CSVs
# sub_csvs = [f for f in all_csvs if "submission" in os.path.basename(f).lower()]
# com_csvs = [f for f in all_csvs if "comment" in os.path.basename(f).lower()]

# # Load and merge all submissions and comments
# if sub_csvs:
#     df_sub = pd.concat([pd.read_csv(f) for f in sub_csvs], ignore_index=True)
# else:
#     df_sub = pd.DataFrame()

# if com_csvs:
#     df_com = pd.concat([pd.read_csv(f) for f in com_csvs], ignore_index=True)
# else:
#     df_com = pd.DataFrame()

# # Filter comments: keep only those with non-empty body and author, and remove automoderator and deleted/removed comments
# mask = df_com['body'].str.len() >= 1
# df_com = df_com.loc[mask]
# df_com['body'] = df_com['body'].astype('str')
# mask = df_com['author'].str.len() >= 1
# df_com = df_com.loc[mask]
# df_com['author'] = df_com['author'].astype('str')

# # Filter comments: keep only those with at least 2 words in the body
# mask = df_com['body'].str.split().apply(len) >= 2
# df_com = df_com.loc[mask]

# # Filter submissions: keep only those with non-empty title, and remove those with title '[deleted by user]'
# mask = df_sub['title'].str.len() >= 1
# df_sub = df_sub.loc[mask]
# df_sub['title'] = df_sub['title'].astype('str')
# df_sub = df_sub[df_sub['title'] != '[deleted by user]']
# df_sub['body'] = df_sub['title']

# # filter out deleted comments and comments by automoderator
# automoderator = df_com['author'].str.lower() == 'automoderator'
# # Use a raw string for the regex pattern to avoid SyntaxWarning
# deleted_com = df_com['body'].str.contains(r'\[deleted\]|\[removed\]', regex=True)
# df_com = df_com[~(automoderator | deleted_com)]

# df_joined = pd.concat([df_sub, df_com])


In [ ]:



# --- Recompute filtered_csv_paths only ---
import os

output_dir = os.path.join('paper4data', 'subreddit_filtered_by_regex')
filtered_csv_paths = [
    os.path.join(output_dir, f)
    for f in os.listdir(output_dir)
    if f.lower().endswith('.csv')
]
# --- Load all filtered CSVs and reconstruct variables as before ---
all_filtered = []
for path in filtered_csv_paths:
    try:
        df = pd.read_csv(path)
        all_filtered.append(df)
    except Exception as e:
        print(f"Error loading filtered csv {path}: {e}")

if all_filtered:
    df_all_filtered = pd.concat(all_filtered, ignore_index=True)
else:
    df_all_filtered = pd.DataFrame()

# Fill empty/null body with title
if not df_all_filtered.empty:
    # Check if both columns exist
    if 'body' in df_all_filtered.columns and 'title' in df_all_filtered.columns:
        # Count how many rows need filling
        mask = df_all_filtered['body'].isna() | (df_all_filtered['body'] == '') | (df_all_filtered['body'].astype(str).str.strip() == '')
        count_to_fill = mask.sum()
        
        # Fill empty/null body with title
        df_all_filtered.loc[mask, 'body'] = df_all_filtered.loc[mask, 'title']
        
        if count_to_fill > 0:
            print(f"Filled {count_to_fill} empty/null body values with title")

# Reconstruct keyword_to_comments (id set for each keyword)
keyword_to_comments = {}
if not df_all_filtered.empty:
    for kw, group in df_all_filtered.groupby('matched_keyword'):
        keyword_to_comments[kw] = set(group['id'].astype(str))


# Reconstruct subreddit_keyword_stats from keyword_to_comments and df_all_filtered (assume both in memory)
# Print overall stats using DataFrame operations
if not df_all_filtered.empty:
    # Get keyword counts using value_counts
    keyword_counts = df_all_filtered.groupby('matched_keyword')['id'].nunique().sort_values(ascending=False)
    print("Total unique comments/submissions matched per keyword:")
    for kw, count in keyword_counts.items():
        print(f"{kw}: {count}")
    
    # Get subreddit x keyword stats using groupby
    print("\nSubreddit x keyword stats (number of unique ids per keyword per subreddit):")
    subreddit_keyword_counts = df_all_filtered.groupby(['subreddit', 'matched_keyword'])['id'].nunique().unstack(fill_value=0)
    for subreddit in subreddit_keyword_counts.index:
        kw_dict = {kw: count for kw, count in subreddit_keyword_counts.loc[subreddit].items() if count > 0}
        # Sort by count descending
        kw_dict_sorted = dict(sorted(kw_dict.items(), key=lambda x: x[1], reverse=True))
        print(f"{subreddit}: {kw_dict_sorted}")
else:
    print("No filtered data available.")

Total unique comments/submissions matched per keyword:
vegan: 2246374
tesla: 453630
veganism: 389782
evs: 231101
vegetarian: 212757
model 3: 86518
plant-based diet: 57631
bev: 55064
phev: 54186
red meat: 49357
tax credit: 49259
electric car: 48245
solar panels: 44011
supercharger: 42813
model y: 39576
rivian: 39295
electric vehicle: 37192
seitan: 35114
charging station: 29103
almond milk: 28022
ioniq 5: 25787
nutritional yeast: 22969
tesla model: 22272
factory farming: 21388
oat milk: 21363
id.4: 19817
e-bike: 18405
charging network: 17847
net metering: 16933
nissan leaf: 13308
lab-grown meat: 13279
beyond meat: 13149
powerwall: 12889
solar power: 11451
impossible burger: 11152
sunpower: 10527
chevy bolt: 10412
all-electric: 10004
tofu scramble: 9690
plug-in hybrid: 9313
sunrun: 8039
range anxiety: 7357
public charger: 7144
level 2 charger: 7138
dairy free: 6881
bike lane: 6529
battery storage: 5691
solar installer: 5362
veganuary: 5190
flexitarian: 5162
home charger: 4901
tofurky: 465

In [6]:
# Create sector mapping from matched_keyword
# Map each keyword to its sector (ignoring _strong/_weak distinction)

# Create reverse mapping: keyword -> sector
keyword_to_sector = {}

# Map transport keywords
for kw in sector_keyword_strength['transport_strong'] + sector_keyword_strength['transport_weak']:
    keyword_to_sector[kw] = 'transport'

# Map housing keywords
for kw in sector_keyword_strength['housing_strong'] + sector_keyword_strength['housing_weak']:
    keyword_to_sector[kw] = 'housing'

# Map food keywords
for kw in sector_keyword_strength['food_strong'] + sector_keyword_strength['food_weak']:
    keyword_to_sector[kw] = 'food'

# Add sector column to df_all_filtered based on matched_keyword
if not df_all_filtered.empty and 'matched_keyword' in df_all_filtered.columns:
    df_all_filtered['sector'] = df_all_filtered['matched_keyword'].map(keyword_to_sector)
    print(f"Added 'sector' column to df_all_filtered")
    print(f"Sector distribution:")
    print(df_all_filtered['sector'].value_counts())
else:
    print("Warning: df_all_filtered is empty or missing 'matched_keyword' column")

Added 'sector' column to df_all_filtered
Sector distribution:
sector
food         3177356
transport    1394862
housing       144775
Name: count, dtype: int64


In [12]:
if 'body' in df_all_filtered.columns and 'title' in df_all_filtered.columns:
    # Count how many rows need filling
    mask = df_all_filtered['body'].isna() | (df_all_filtered['body'] == '') | (df_all_filtered['body'].astype(str).str.strip() == '')
    count_to_fill = mask.sum()
    
    # Fill empty/null body with title
    df_all_filtered.loc[mask, 'body'] = df_all_filtered.loc[mask, 'title']
    
    if count_to_fill > 0:
        print(f"Filled {count_to_fill} empty/null body values with title")


Filled 319862 empty/null body values with title


In [14]:
df_all_filtered.sector

0          housing
1             food
2             food
3             food
4             food
            ...   
4716988       food
4716989       food
4716990       food
4716991       food
4716992       food
Name: sector, Length: 4716993, dtype: object

In [ ]:
# # First time run - process all keywords
# all_comments = df_topics_all['Document'].tolist()
# sector_candidates, keyword_comment_map = filter_sector_comments(all_comments, sector_keywords)
# pd.to_pickle(sector_candidates, results_file)
# pd.to_pickle(keyword_comment_map, keyword_map_file)
# pd.to_pickle(set(sector_keywords), keyword_list_file)

In [4]:
# Visualization 4: Wordmaps for each sector (all in one figure)
from wordcloud import WordCloud
import matplotlib.pyplot as plt
import numpy as np

fig, axes = plt.subplots(3, 1, figsize=(10, 12))

def get_sequential_palette_for_sector(sector, n_labels):
    sector_cmaps = {
        'transport': 'Reds',
        'housing': 'Oranges',
        'food': 'Blues'
    }
    cmap_name = sector_cmaps.get(sector, 'Greys')
    cmap = plt.get_cmap(cmap_name)
    colors = [cmap(x) for x in np.linspace(0.4, 2, n_labels)][::-1]
    return colors

def scale_font_sizes(freq_dict, min_font=8, max_font=50):
    """
    Scale frequencies to font sizes between min_font and max_font.
    All words are included, even those with lowest frequency.
    """
    if not freq_dict:
        return {}
    values = list(freq_dict.values())
    min_freq = min(values)
    max_freq = max(values)
    # Avoid division by zero if all frequencies are the same
    if max_freq == min_freq:
        return {k: (min_font + max_font) // 2 for k in freq_dict}
    scaled = {}
    for k, v in freq_dict.items():
        # Linear scaling
        size = min_font + (v - min_freq) * (max_font - min_font) / (max_freq - min_freq)
        scaled[k] = size
    return scaled

def mpl_color_to_pil(color):
    """
    Convert a matplotlib RGBA color (floats 0-1) to a PIL-compatible RGB or RGBA tuple (ints 0-255).
    """
    if isinstance(color, tuple):
        if len(color) == 4:
            return tuple(int(round(255 * c)) for c in color[:3])  # ignore alpha for PIL
        elif len(color) == 3:
            return tuple(int(round(255 * c)) for c in color)
    # fallback to black
    return (0, 0, 0)

# Create sector mapping from sector_keyword_strength
sector_keywords_map = {
    'transport': sector_keyword_strength['transport_strong'] + sector_keyword_strength['transport_weak'],
    'housing': sector_keyword_strength['housing_strong'] + sector_keyword_strength['housing_weak'],
    'food': sector_keyword_strength['food_strong'] + sector_keyword_strength['food_weak']
}
sectors_list = ['transport', 'housing', 'food']

for idx, sector in enumerate(sectors_list):
    # Get keywords for this sector from sector_keywords_map
    sector_keywords = sector_keywords_map[sector]
    
    if len(sector_keywords) == 0:
        # No keywords for this sector
        axes[idx].text(0.5, 0.5, f'No keywords found for {sector}', 
                      ha='center', va='center', transform=axes[idx].transAxes)
        axes[idx].axis('off')
        continue
    
    # Filter keyword_counts to only include keywords from this sector
    sector_counts = keyword_counts[keyword_counts.index.isin(sector_keywords)]
    
    # Filter out zero counts
    non_zero_keywords = sector_counts[sector_counts > 0]
    
    if len(non_zero_keywords) == 0:
        # No non-zero keywords for this sector
        axes[idx].text(0.5, 0.5, f'No non-zero keywords found for {sector}', 
                      ha='center', va='center', transform=axes[idx].transAxes)
        axes[idx].axis('off')
        continue
    
    # Sort by frequency
    sorted_keywords = dict(sorted(non_zero_keywords.items(), key=lambda x: x[1], reverse=True))
    
    # Prepare wordcloud frequencies, format as "word (n=xx)"
    wc_freq = {f"{k} (n={v})": v for k, v in sorted_keywords.items() if v > 0}
    
    # Scale font sizes for all words
    font_sizes = scale_font_sizes(wc_freq, min_font=8, max_font=40)
    
    # Use a color map for each sector using the helper function
    cmap_colors = get_sequential_palette_for_sector(sector, len(wc_freq))
    
    # Assign a color to each word based on its rank, and convert to PIL-compatible
    word_color_map = {}
    for i, word in enumerate(wc_freq.keys()):
        word_color_map[word] = mpl_color_to_pil(cmap_colors[i % len(cmap_colors)])
    
    # Custom color function to use the mapped color
    def color_func(word, font_size, position, orientation, random_state=None, **kwargs):
        return word_color_map.get(word, (0, 0, 0))

    # Generate wordcloud
    wc = WordCloud(width=800, height=200, background_color='white',
                   color_func=color_func, prefer_horizontal=.6, 
                   collocations=False)
    wc.generate_from_frequencies(wc_freq)

    # Now, update font sizes in layout_ directly
    new_layout = []
    for (word, count), font_size, position, orientation, color in wc.layout_:
        # Use our scaled font size
        scaled_size = font_sizes.get(word, 8)
        new_layout.append(((word, count), scaled_size, position, orientation, color))
    wc.layout_ = new_layout

    # Draw wordcloud
    axes[idx].imshow(wc.to_array(), interpolation='bilinear')
    axes[idx].axis('off')
    axes[idx].set_title(f"{sector.capitalize()} (n={len(non_zero_keywords)} keywords)", fontsize=16)

plt.tight_layout()
plt.savefig('paper4figs/fig1_sector_wordmaps.png', dpi=300, bbox_inches='tight')
plt.savefig('paper4figs/fig1_sector_wordmaps.pdf', bbox_inches='tight')
plt.close()

# FasTopic

# plot wordmaps

# GPT based training , validation and test data

In [ ]:
# Import the functions
import paper4_GPT_arguments
import json
from importlib import reload
from paper4_GPT_arguments import main, process_single_sector

# Reload the module
reload(paper4_GPT_arguments)

# Define ranges for validation data
ranges = [
    (4000, 4150),
    (4150, 4300)
]

# Process each validation range
for i, (start, end) in enumerate(ranges, start=1):
    # Sample comments from each sector
    sampled_comments = {
        sector: list(comments)[start:end] if len(comments) > end else list(comments)
        for sector, comments in comments_by_sector.items()
    }

    # Process with parallel agents
    results, results_df, sector_labels = main(
        comments_by_sector=sampled_comments,
        num_runs=1,
        max_comments_per_sector=1000,
        num_agents=3,
        batch_size=1,
        use_survey_frames=False
    )

    # Create validation data dictionary
    validation_data = {}
    for sector in sector_labels:
        sector_df = results_df[results_df['sector'] == sector]
        validation_data[sector] = {}
        
        for idx, row in sector_df.iterrows():
            comment = row['comment']
            labels = []
            for label in sector_labels[sector]:
                col = f"{sector}_{label.lower().replace(' ', '_')}"
                if row[col] > 0:
                    labels.append(label)
            if labels:  # Only add if comment has labels
                validation_data[sector][comment] = labels

    # Save results with index as validation data
    results_filename = f"paper4data/validation_data_by_GPT_results_{i}.json"
    validation_data_filename = f"paper4data/validation_data_by_GPT_dict_{i}.json"

    with open(results_filename, 'w') as f:
        json.dump(results, f, indent=2)
        
    with open(validation_data_filename, 'w') as f:
        json.dump(validation_data, f, indent=2)


In [ ]:
#     # Print overall statistics first
# print("=== Overall Statistics ===")
# for sector in sector_labels.keys():
#     sector_df = results_df[results_df['sector'] == sector].copy()
#     total_comments = len(sector_df)
#     print(f"\n{sector.upper()} SECTOR:")
#     print(f"Total comments: {total_comments}")
    
#     # Count unlabeled comments (including newly added ones)
#     no_labels = sector_df.copy()
#     try:
#         for label in sector_labels[sector]:
#             col_name = f"{sector}_{label.lower().replace(' ', '_')}"
#             no_labels = no_labels[no_labels[col_name] == 0]
#         unlabeled_count = len(no_labels)
#         print(f"Unlabeled comments: {unlabeled_count}")
        
#         # Count comments per label
#         print("Comments per label:")
#         for label in sector_labels[sector]:
#             label_col = f"{sector}_{label.lower().replace(' ', '_')}"
#             labeled_count = len(sector_df[sector_df[label_col] > 0])
#             print(f"- {label}: {labeled_count}")
#     except KeyError as e:
#         print(f"Warning: Could not process labels for sector {sector}. Missing column: {str(e)}")
#     print("-" * 50)

# # Create single output file for all sectors
# with open(f'paper4data/gpt3_check_all_sectors.txt', 'w', encoding='utf-8') as f:
#     # Add meta description at top
#     f.write("=" * 80 + "\n")
#     f.write("FILE CONTENTS DESCRIPTION:\n")
#     f.write("This file contains all comments from all sectors with their applied labels\n")
#     f.write("Format: [Label1, Label2, ...] Comment text\n")
#     f.write("Comments longer than 4000 characters are truncated with '...'\n")
#     f.write("=" * 80 + "\n\n")
    
#     # Process each sector
#     for sector in sector_labels.keys():
#         f.write(f"\n{sector.upper()} SECTOR COMMENTS:\n")
#         f.write("=" * 80 + "\n\n")
        
#         sector_df = results_df[results_df['sector'] == sector].copy()
        
#         # Get all comments for this sector
#         for _, row in sector_df.iterrows():
#             try:
#                 # Collect all labels that apply to this comment
#                 applied_labels = []
#                 for label in sector_labels[sector]:
#                     col_name = f"{sector}_{label.lower().replace(' ', '_')}"
#                     if row[col_name] > 0:
#                         applied_labels.append(label)
                
#                 # Format comment text
#                 comment_text = row['comment'][:4000] + "..." if len(row['comment']) > 4000 else row['comment']
                
#                 # Write comment with all its labels in one bracket
#                 label_text = ", ".join(applied_labels) if applied_labels else "NO LABELS"
#                 f.write(f"[{label_text}] {comment_text}\n\n")
#             except KeyError as e:
#                 print(f"Warning: Could not process comment in sector {sector}. Missing column: {str(e)}")
#                 continue

# finetune Roberta

In [ ]:
# !python paper4_BERT_finetuning.py

## optimal thesholds: probably overfitting

In [ ]:
# !python paper4_multilabel_threshold_optimizer.py

# Dendogram

In [ ]:
# import numpy as np
# import pandas as pd

# def label_cooccurrence_matrix(all_true_labels, label_names):
#     """
#     all_true_labels: list of binary lists (len = num_labels)
#     label_names: list of labels, len = num_labels
#     returns: 
#       - co_occur: DataFrame count of how often i&j both = 1
#       - jaccard: Jaccard similarity matrix
#       - cond_prob: P(j|i) = co_occur[i,j] / total_i
#     """
#     Y = np.array(all_true_labels)  # shape (N, L)
#     L = Y.shape[1]
#     co = np.dot(Y.T, Y)            # co[i,j] = #samples where both label i and j are 1
#     total = Y.sum(axis=0)          # total[i] = #samples where label i is 1

#     # build DataFrames
#     co_occur = pd.DataFrame(co, index=label_names, columns=label_names)
    
#     # Calculate Jaccard similarity
#     # For i=j, intersection (co_occur) equals union (total), so jaccard = 1
#     # For i≠j, jaccard = intersection / union = co_occur / (total_i + total_j - co_occur)
#     union = total[:,None] + total[None,:] - co_occur
#     jaccard = co_occur / union
    
#     # Calculate conditional probability, avoiding division by zero
#     total_clipped = np.maximum(total[:,None], 1)  # Ensure denominator is at least 1
#     cond_prob = co_occur / total_clipped   # P(j|i)
    
#     return co_occur, jaccard, cond_prob

# # Calculate and save co-occurrence matrices for each sector
# for sector, model_info in all_sector_models.items():
#     print(f"\nAnalyzing label co-occurrences for {sector} sector:")
    
#     # Get true labels and label names for this sector
#     sector_data = all_data[sector]
#     label_names = model_info['label_names']
    
#     # Convert true labels to binary format
#     true_labels_list = []
#     for true_labels in sector_data.values():
#         binary_labels = [1 if label in true_labels else 0 for label in label_names]
#         true_labels_list.append(binary_labels)
    
#     # Calculate matrices
#     co_occur, jaccard, cond_prob = label_cooccurrence_matrix(true_labels_list, label_names)
    
#     # Save matrices to Excel files
#     jaccard.to_excel(f'paper4data/eval_class_overlaps_{sector}_jaccard.xlsx')
#     cond_prob.to_excel(f'paper4data/eval_class_overlaps_{sector}_condprob.xlsx')
    
#     print(f"Saved matrices to paper4data/eval_class_overlaps_{sector}_*.xlsx")



Analyzing label co-occurrences for transport sector:
Saved matrices to paper4data/eval_class_overlaps_transport_*.xlsx

Analyzing label co-occurrences for housing sector:
Saved matrices to paper4data/eval_class_overlaps_housing_*.xlsx

Analyzing label co-occurrences for food sector:
Saved matrices to paper4data/eval_class_overlaps_food_*.xlsx


In [ ]:
# # Analyze label relationships across sectors
# from scipy.cluster.hierarchy import dendrogram, linkage
# import matplotlib.pyplot as plt
# from scipy.spatial.distance import squareform

# def analyze_label_relationships(sector_models, all_data):
#     """Analyze relationships between labels using lift and hierarchical clustering"""
#     print("\nAnalyzing label relationships across sectors...")
    
#     for sector, model_info in sector_models.items():
#         print(f"\n--- {sector.upper()} SECTOR ---")
        
#         # Get true labels and label names
#         sector_data = all_data[sector]
#         label_names = model_info['label_names']
        
#         # Convert to binary format
#         true_labels_list = []
#         for true_labels in sector_data.values():
#             binary_labels = [1 if label in true_labels else 0 for label in label_names]
#             true_labels_list.append(binary_labels)
            
#         # Calculate co-occurrence matrix
#         Y = np.array(true_labels_list)
#         N = Y.shape[0]  # Total number of samples
#         co = np.dot(Y.T, Y)  # Co-occurrence counts
#         marginal = Y.sum(axis=0)  # Marginal counts
        
#         # Calculate expected co-occurrence under independence
#         expected = np.outer(marginal, marginal) / N
        
#         # Calculate lift matrix
#         lift = co / expected
#         lift = pd.DataFrame(lift, index=label_names, columns=label_names)
        
#         # Save lift matrix
#         lift.to_excel(f'paper4data/eval_class_overlaps_{sector}_lift.xlsx')
        
#         # Convert lift to distance (1/lift, capped at 1 for lift > 1)
#         dist_lift = 1 / np.maximum(lift, 1)
#         np.fill_diagonal(dist_lift.values, 0)  # Set diagonal to 0
        
#         # Perform hierarchical clustering
#         condensed_dist = squareform(dist_lift)
#         Z = linkage(condensed_dist, method='average')
        
#         # Plot dendrogram
#         plt.figure(figsize=(10, 7))
#         dendrogram(Z, labels=label_names, leaf_rotation=90)
#         plt.title(f"{sector} - Clustering by Lift")
#         plt.tight_layout()
#         plt.savefig(f'paper4data/clustering_{sector}_lift.png')
#         plt.close()
        
#         print(f"Saved lift matrix: paper4data/eval_class_overlaps_{sector}_lift.xlsx")
#         print(f"Saved clustering plot: paper4data/clustering_{sector}_lift.png")
        
#         # Print strongly associated pairs (lift >= 2)
#         strong_pairs = []
#         for i in range(len(label_names)):
#             for j in range(i+1, len(label_names)):
#                 if lift.iloc[i,j] >= 2:
#                     strong_pairs.append((label_names[i], label_names[j], lift.iloc[i,j]))
        
#         if strong_pairs:
#             print("\nStrongly associated label pairs (lift >= 2):")
#             for l1, l2, lift_val in sorted(strong_pairs, key=lambda x: x[2], reverse=True):
#                 print(f"{l1} - {l2}: {lift_val:.2f}")

# # Run analysis
# analyze_label_relationships(all_sector_models, all_data)



Analyzing label relationships across sectors...

--- TRANSPORT SECTOR ---
Saved lift matrix: paper4data/eval_class_overlaps_transport_lift.xlsx
Saved clustering plot: paper4data/clustering_transport_lift.png

Strongly associated label pairs (lift >= 2):
Depreciation And Resale - Second Hand Market: 12.50
Battery Cost - Reliability: 8.93
Depreciation And Resale - Purchase Price: 5.62
Equity And Accessibility - Second Hand Market: 5.51
Battery Recycling And End Of Life - Depreciation And Resale: 5.00
Fun To Drive - Running Costs: 4.69
Depreciation And Resale - Equity And Accessibility: 4.41
Battery Recycling And End Of Life - Mineral Supply Chain: 4.27
Purchase Price - Second Hand Market: 3.91
Range Anxiety - Reliability: 3.49
Reliability - Running Costs: 3.35
Equity And Accessibility - Purchase Price: 3.31
Purchase Price - Range Anxiety: 3.26
Fun To Drive - Second Hand Market: 3.12
Battery Recycling And End Of Life - Lifecycle Emissions: 2.94
Fun To Drive - Technology Roadmap: 2.88
Cha

Saved lift matrix: paper4data/eval_class_overlaps_housing_lift.xlsx
Saved clustering plot: paper4data/clustering_housing_lift.png

Strongly associated label pairs (lift >= 2):
Community Engagement And Nimby - Permitting And Bureaucracy: 46.00
Community Engagement And Nimby - Wildlife: 19.71
Permitting And Bureaucracy - Wildlife: 13.14
Property Value Impact - Subsidy And Tariff Debate: 7.89
Decommissioning And Waste - Visual Impact: 5.11
Community Engagement And Nimby - Subsidy And Tariff Debate: 3.94
Green Jobs And Workforce - Local Economy: 3.76
Land Use - Wildlife: 2.72

--- FOOD SECTOR ---
Saved lift matrix: paper4data/eval_class_overlaps_food_lift.xlsx
Saved clustering plot: paper4data/clustering_food_lift.png

Strongly associated label pairs (lift >= 2):
Cost - Farmer Livelihoods: 4.05
Health - Health Deficiency Anxiety: 3.47
Farmer Livelihoods - Systemic Vs Individual Action: 3.10
Cultural Identity And Tradition - Health Deficiency Anxiety: 2.86
Animal Welfare - Social Media Infl

## testing evaluating on new data

In [ ]:
# !python evaluate_trained_models.py

=== TESTING SECTOR MODELS WITH TOP-K ACCURACY ===

--- TRANSPORT SECTOR ---


Total test samples: 378
Top-3 accuracy: 70.37% (266/378)

Example predictions:

Example 1:
Comment: With today's announcement, Plug Power has established the world's first ever Gigafactory for Proton ...
True labels: ['Technology Roadmap']
Top-3 predictions: ['Mineral Supply Chain', 'Policy And Mandates', 'Lifecycle Emissions']
Hit: ✗

Example 2:
Comment: ###[Transition to Electric Vehicles They Say. With Both Major Parties Signing Up to the United Natio...
True labels: ['Grid Impact And Energy Mix', 'Policy And Mandates', 'Charging Infrastructure']
Top-3 predictions: ['Policy And Mandates', 'Charging Infrastructure', 'Purchase Price']
Hit: ✓

Example 3:
Comment: &gt; It's hard to tell right now if this facility will be utilizing these latest discoveries or will...
True labels: ['Technology Roadmap']
Top-3 predictions: ['Alternative Modes', 'Environmental Benefit', 'Mineral Supply Chain']
Hit: ✗

--- HOUSING SECTOR ---
Total test samples: 282
Top-3 accuracy: 76.95% (217/282)

Example p

## Raw hypothesis counts to remove hypthesis

In [ ]:
# !python analyze_hypothesis_counts.py

Loaded and merged test data for 3 sectors

EVALUATING ON TEST DATA WITH OPTIMAL THRESHOLDS

--- TRANSPORT SECTOR ---
Number of test comments: 378


Getting predictions:   1%|          | 4/378 [00:00<00:24, 15.10it/s]


Micro-averaged Jaccard Index: 0.161

Alternative Modes (appears in 47 test comments):
Jaccard Index: 0.179
Threshold: 0.27

Battery Cost (appears in 12 test comments):
Jaccard Index: 0.067
Threshold: 0.14

Battery Recycling And End Of Life (appears in 7 test comments):
Jaccard Index: 0.000
Threshold: 0.12

Charging Infrastructure (appears in 77 test comments):
Jaccard Index: 0.270
Threshold: 0.28

Depreciation And Resale (appears in 3 test comments):
Jaccard Index: 0.000
Threshold: 0.11

Environmental Benefit (appears in 55 test comments):
Jaccard Index: 0.309
Threshold: 0.31

Equity And Accessibility (appears in 24 test comments):
Jaccard Index: 0.028
Threshold: 0.13

Fun To Drive (appears in 10 test comments):
Jaccard Index: 0.000
Threshold: 0.10

Grid Impact And Energy Mix (appears in 51 test comments):
Jaccard Index: 0.156
Threshold: 0.19

Insurance And Ownership Risk (appears in 2 test comments):
Jaccard Index: 0.000
Threshold: 0.58

Lifecycle Emissions (appears in 28 test commen


Micro-averaged Jaccard Index: 0.163

Community Engagement And Nimby (appears in 5 test comments):
Jaccard Index: 0.000
Threshold: 0.69

Cost Volatility And Economic Risk (appears in 23 test comments):
Jaccard Index: 0.088
Threshold: 0.20

Decommissioning And Waste (appears in 30 test comments):
Jaccard Index: 0.244
Threshold: 0.26

Foreign Dependence And Trade (appears in 39 test comments):
Jaccard Index: 0.188
Threshold: 0.35

Green Jobs And Workforce (appears in 14 test comments):
Jaccard Index: 0.000
Threshold: 0.11

Grid Stability And Storage (appears in 85 test comments):
Jaccard Index: 0.447
Threshold: 0.36

Land Use (appears in 25 test comments):
Jaccard Index: 0.156
Threshold: 0.20

Local Economy (appears in 73 test comments):
Jaccard Index: 0.165
Threshold: 0.36

Permitting And Bureaucracy (appears in 4 test comments):
Jaccard Index: 0.000
Threshold: 0.24

Property Value Impact (appears in 4 test comments):
Jaccard Index: 0.000
Threshold: 0.41

Resilience And Blackout Protect


Micro-averaged Jaccard Index: 0.421

Animal Welfare (appears in 58 test comments):
Jaccard Index: 0.309
Threshold: 0.30

Cost (appears in 27 test comments):
Jaccard Index: 0.000
Threshold: 0.32

Cultural Identity And Tradition (appears in 16 test comments):
Jaccard Index: 0.000
Threshold: 0.17

Environmental Impact (appears in 247 test comments):
Jaccard Index: 0.764
Threshold: 0.60

Farmer Livelihoods (appears in 7 test comments):
Jaccard Index: 0.000
Threshold: 0.18

Health (appears in 51 test comments):
Jaccard Index: 0.227
Threshold: 0.45

Health Deficiency Anxiety (appears in 20 test comments):
Jaccard Index: 0.250
Threshold: 0.32

Lab Grown And Alt Proteins (appears in 32 test comments):
Jaccard Index: 0.262
Threshold: 0.31

Psychology And Identity (appears in 54 test comments):
Jaccard Index: 0.238
Threshold: 0.42

Social Media Influence (appears in 13 test comments):
Jaccard Index: 0.000
Threshold: 0.15

Systemic Vs Individual Action (appears in 79 test comments):
Jaccard Inde

## Test sector models with dynamic top-k accuracy (n where n is number of true labels)


In [ ]:
# def test_sector_models_dynamic_top_k(sector_models, test_data):
#     """Test sector models with dynamic top-k accuracy where k = number of true labels"""
#     print("=== TESTING SECTOR MODELS WITH DYNAMIC TOP-K ACCURACY ===")
    
#     for sector, model_data in sector_models.items():
#         if sector not in test_data:
#             print(f"No test data for {sector} sector")
#             continue
            
#         print(f"\n--- {sector.upper()} SECTOR ---")
#         model = model_data['model']
#         tokenizer = model_data['tokenizer']
#         label_names = model_data['label_names']
        
#         device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#         model.to(device)
#         model.eval()
        
#         sector_test_data = test_data[sector]
#         total_samples = 0
#         dynamic_hits = 0
        
#         for comment, true_labels in sector_test_data.items():
#             if not true_labels or not comment.strip():
#                 continue
                
#             total_samples += 1
            
#             # Dynamic k based on number of true labels
#             k = len(true_labels)
            
#             # Tokenize and predict
#             inputs = tokenizer(comment, return_tensors='pt', truncation=True, 
#                              padding=True, max_length=512)
#             inputs = {k: v.to(device) for k, v in inputs.items()}
            
#             with torch.no_grad():
#                 outputs = model(**inputs)
#                 # Handle different model output formats
#                 if hasattr(outputs, 'logits'):
#                     logits = outputs.logits
#                 else:
#                     # If outputs is a tensor directly
#                     logits = outputs
#                 probabilities = torch.sigmoid(logits).cpu().numpy()[0]
            
#             # Get top-k predictions where k = number of true labels
#             top_k_indices = np.argsort(probabilities)[-k:][::-1]
#             top_k_labels = [label_names[i] for i in top_k_indices]
            
#             # Check if any true label is in top-k predictions
#             if any(true_label in top_k_labels for true_label in true_labels):
#                 dynamic_hits += 1
        
#         # Calculate dynamic top-k accuracy
#         if total_samples > 0:
#             dynamic_accuracy = (dynamic_hits / total_samples) * 100
#             print(f"Total test samples: {total_samples}")
#             print(f"Dynamic top-n accuracy: {dynamic_accuracy:.2f}% ({dynamic_hits}/{total_samples})")
#         else:
#             print(f"No valid test samples for {sector} sector")

# # Test with dynamic top-k accuracy
# test_sector_models_dynamic_top_k(all_sector_models, all_test_data)
# # test_sector_models_with_top_k(all_sector_models, all_test_data, k=1)

# checking with GPT o3

In [ ]:
# !python generate_detailed_test_results.py

# Plot Roberta : EVs, solar, vegetarian


In [ ]:
from evaluate_trained_models import (
    load_trained_model, 
    load_test_data, 
    predict_with_scores
)
def get_model_predictions(comments_by_sector, 
                         save_dir="paper4data/sectorwise_roberta_classifications",
                         thresholded_save_dir="paper4data/sectorwise_roberta_classifications_thresholded"):
    """Get model predictions for a sample of 100 comments per sector, save raw and thresholded results to disk, one CSV per sector"""
    
    import os
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    
    # Find and load all trained models
    model_dirs = glob.glob("models/top7_*_distilbert_base_uncased")
    
    if not model_dirs:
        print("No trained models found!")
        return None, None
    
    # Load all sector models
    all_sector_models = {}
    optimal_thresholds = {}
    sector_labels = {}
    
    for model_dir in model_dirs:
        # Extract sector from directory name
        sector = model_dir.split('_')[1]  # top7_SECTOR_model
        
        print(f"Loading {sector.upper()} model...")
        
        # Load model
        model, tokenizer, label_names, sector_optimal_thresholds = load_trained_model(model_dir, device)
        
        if model is None:
            print(f"Failed to load {sector} model")
            continue
            
        all_sector_models[sector] = {
            'model': model,
            'tokenizer': tokenizer,
            'label_names': label_names
        }
        
        optimal_thresholds[sector] = sector_optimal_thresholds
        sector_labels[sector] = label_names
    
    if not all_sector_models:
        print("No models loaded successfully!")
        return None, None
        
    # Ensure output directories exist
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(thresholded_save_dir, exist_ok=True)
    
    # Store all results for return
    all_results = []
    all_thresholded_results = []
    
    # Process each sector, only first 100 comments
    for sector, comments in comments_by_sector.items():
        if sector not in all_sector_models:
            continue
            
        print(f"\nProcessing {sector} comments (sampling 100)...")
        
        model = all_sector_models[sector]['model']
        tokenizer = all_sector_models[sector]['tokenizer']
        label_names = all_sector_models[sector]['label_names']
        
        # Sample up to 100 comments
        sample_comments = list(comments)[:]
        
        # Get predictions for each comment
        sector_results = []
        sector_thresholded_results = []
        for text in tqdm(sample_comments):
            # Get predictions
            prediction_results = predict_with_scores(text, model, tokenizer, label_names, device)
            scores = [res['score'] for res in prediction_results]
            
            # Create result dict
            result = {'sector': sector, 'text': text}
            
            # Add scores for each label
            for label, score in zip(label_names, scores):
                result[f"{sector}_{label}"] = score
                
            sector_results.append(result)
        
        # Save raw results for this sector
        sector_df = pd.DataFrame(sector_results)
        sector_csv_path = os.path.join(save_dir, f"{sector}_Roberta_classifications.csv")
        sector_df.to_csv(sector_csv_path, index=False)
        print(f"Saved {sector} Roberta classifications to {sector_csv_path}")
        all_results.extend(sector_results)
        
        # Now create thresholded version for this sector
        thresholds = optimal_thresholds.get(sector, {})
        for row in sector_results:
            thresholded_row = {'sector': sector, 'text': row['text']}
            for label in label_names:
                col = f"{sector}_{label}"
                score = row[col]
                # Use optimal threshold if available, else 0.5
                threshold = thresholds.get(label, 0.5)
                thresholded_row[col] = int(score > threshold)
            sector_thresholded_results.append(thresholded_row)
        thresholded_sector_df = pd.DataFrame(sector_thresholded_results)
        thresholded_sector_csv_path = os.path.join(thresholded_save_dir, f"{sector}_Roberta_classifications_thresholded.csv")
        thresholded_sector_df.to_csv(thresholded_sector_csv_path, index=False)
        print(f"Saved thresholded {sector} Roberta classifications to {thresholded_sector_csv_path}")
        all_thresholded_results.extend(sector_thresholded_results)
    
    # Optionally, return all results as a single dataframe for further use
    results_df = pd.DataFrame(all_results)
    thresholded_df = pd.DataFrame(all_thresholded_results)
    
    return results_df, sector_labels

# Generate predictions and plotting dataframe, and save results
# results_df, sector_labels = get_model_predictions(comments_by_sector)

NameError: name 'torch' is not defined

In [11]:
import os
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt

def plot_sector_results_from_thresholded_csvs(
    sector_labels, 
    data_dir=r"C:\Users\chowdhary\Desktop\Redit project 4.09.2024\paper4data\sectorwise_roberta_classifications_thresholded",
    comments_by_sector=None
):
    """
    Loads thresholded, binarized sector classification CSVs and plots the percentage of comments
    for each label in each sector. The denominator for percentage is the total number of comments
    in the sector (from comments_by_sector), not just those in the CSV (which may only include labelled).
    Plots all three sectors in a single figure with three subplots.
    """
    if comments_by_sector is None:
        raise ValueError("comments_by_sector must be provided to get total counts per sector.")

    # Mapping sector to intervention name for display
    sector_intervention = {
        'transport': 'EVs',
        'housing': 'Solar',
        'food': 'Vegetarianism'
    }
    FONT_SIZE = 17
    fig = plt.figure(figsize=(12, 24))
    gs = fig.add_gridspec(3, 1, height_ratios=[1, 1, 1], hspace=0.3)
    axes = []
    for idx, (sector, labels) in enumerate(sector_labels.items()):
        ax = fig.add_subplot(gs[idx, 0])
        axes.append(ax)
        label_counts = {}

        # Load thresholded, binarized CSV for this sector
        csv_path = os.path.join(data_dir, f"{sector}_Roberta_classifications_thresholded.csv")
        if not os.path.exists(csv_path):
            print(f"File not found: {csv_path}")
            continue
        sector_df = pd.read_csv(csv_path)

        # Get total number of comments in this sector from comments_by_sector
        if sector not in comments_by_sector:
            print(f"Sector '{sector}' not found in comments_by_sector.")
            total = 0
        else:
            total = len(comments_by_sector[sector])

        for label in labels:
            colname = f"{sector}_{label}"
            if colname in sector_df.columns:
                # Count number of comments where label is present (==1)
                label_counts[label] = int((sector_df[colname] == 1).sum())
            else:
                label_counts[label] = 0  # If column missing, count as 0

        # Sort labels by frequency (descending)
        labels_and_counts = [(label, label_counts[label]) for label in labels]
        labels_and_counts.sort(key=lambda x: x[1], reverse=True)
        sorted_labels = [label for label, count in labels_and_counts]
        sorted_freqs = [count for label, count in labels_and_counts]

        # Avoid division by zero
        if total > 0:
            sorted_pcts = [count / total * 100 for count in sorted_freqs]
        else:
            sorted_pcts = [0 for _ in sorted_freqs]

        y_pos = np.arange(len(sorted_labels))
        color = '#43AA8B' if sector == 'transport' else '#F94144' if sector == 'food' else '#FFB703'
        ax.barh(y_pos, sorted_pcts, color=color, alpha=0.8, height=0.6)
        ax.set_yticks(y_pos)
        ax.set_yticklabels([label.lower() for label in sorted_labels], fontsize=FONT_SIZE * 1.54)
        ax.set_xlabel('% of comments', fontsize=FONT_SIZE * 1.54, labelpad=10)
        ax.tick_params(axis='x', labelsize=FONT_SIZE * 1.54)
        # Compose sector title with intervention
        intervention = sector_intervention.get(sector, "")
        if intervention:
            sector_title = f"{sector.capitalize()}\n({intervention})"
        else:
            sector_title = sector.capitalize()
        # Set title location to x=0.6, y=0.8
        ax.set_title(
            sector_title,
            fontsize=FONT_SIZE * 1.5,
            pad=20,
            loc='left',
            rotation=0,
            x=0.8,
            y=0.75,
            bbox=dict(facecolor='white', edgecolor='black', boxstyle='round,pad=0.1')
        )
        print(f"Plotting sector: {sector_title}")  # Print sector name with intervention
        for i, (pct, count) in enumerate(zip(sorted_pcts, sorted_freqs)):
            ax.text(pct + 1, i, f'{pct:.1f}% (n={count})', va='center',
                   fontsize=FONT_SIZE * 1.32, color='#444444')
                   
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.grid(False)
        ax.set_axisbelow(True)
    # Save the single figure with all three sectors
    os.makedirs('paper4figs', exist_ok=True)
    plt.savefig(f'paper4figs/paper4_fig2_allsectors.pdf', bbox_inches='tight', dpi=300)
    plt.savefig(f'paper4figs/paper4_fig2_allsectors.png', bbox_inches='tight', dpi=300)
    plt.close()

# You must now pass comments_by_sector as an argument!
plot_sector_results_from_thresholded_csvs(sector_labels, comments_by_sector=comments_by_sector)

Plotting sector: Food
(Vegetarianism)
Plotting sector: Housing
(Solar)
Plotting sector: Transport
(EVs)


In [ ]:
# Get model directories as in get_model_predictions
import glob
import torch

def get_sector_labels_from_models():
    """
    Finds all model directories and loads each model to extract its label names.
    Returns a dict: {sector: [label1, label2, ...], ...}
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    # Find all trained model directories
    model_dirs = glob.glob("models/top7_*_distilbert_base_uncased")
    sector_labels = {}
    for model_dir in model_dirs:
        # Extract sector from directory name
        sector = model_dir.split('_')[1]  # expects format: top7_SECTOR_model
        # Load model, tokenizer, label_names, thresholds
        model, tokenizer, label_names, _ = load_trained_model(model_dir, device)
        if label_names is not None:
            sector_labels[sector] = label_names
    return sector_labels

sector_labels = get_sector_labels_from_models()
sector_labels

{'food': ['Animal Welfare',
  'Environmental Impact',
  'Health',
  'Lab Grown And Alt Proteins',
  'Psychology And Identity',
  'Systemic Vs Individual Action',
  'Taste And Convenience'],
 'housing': ['Decommissioning And Waste',
  'Foreign Dependence And Trade',
  'Grid Stability And Storage',
  'Land Use',
  'Local Economy',
  'Subsidy And Tariff Debate',
  'Utility Bills'],
 'transport': ['Alternative Modes',
  'Charging Infrastructure',
  'Environmental Benefit',
  'Grid Impact And Energy Mix',
  'Mineral Supply Chain',
  'Policy And Mandates',
  'Purchase Price']}

In [12]:
for sector, comments in comments_by_sector.items():
    print(f"Total size for sector '{sector}': {len(comments)}")


Total size for sector 'transport': 14924
Total size for sector 'housing': 9388
Total size for sector 'food': 8414


# Plot Temporal Roberta : EVs, solar, vegetarian


In [25]:
  # --- Load binarized data with optimal thresholds, merge with created_utc_, and plot temporal trends (high-aesthetic stacked area charts, 3-month rolling average) ---
import matplotlib.pyplot as plt
import numpy as np
import os
import seaborn as sns
import matplotlib.ticker as mticker
import matplotlib.colors as mcolors

from adjustText import adjust_text

plt.style.use('seaborn-v0_8-white')
sns.set_context("notebook", font_scale=1.0)
sns.set_palette("Set2")

binarized_data_dir = r"C:\Users\chowdhary\Desktop\Redit project 4.09.2024\paper4data\sectorwise_roberta_classifications_thresholded"

sector_dfs = []
for sector in sector_labels.keys():
    csv_path = os.path.join(binarized_data_dir, f"{sector}_Roberta_classifications_thresholded.csv")
    if not os.path.exists(csv_path):
        print(f"File not found: {csv_path}")
        continue
    df = pd.read_csv(csv_path)
    df['sector'] = sector
    sector_dfs.append(df)
binarized_df = pd.concat(sector_dfs, ignore_index=True)

if 'Document' not in binarized_df.columns and 'text' in binarized_df.columns:
    binarized_df = binarized_df.rename(columns={'text': 'Document'})
merged_df = pd.merge(
    binarized_df,
    df_joint_w_topics[['Document', 'created_utc_']],
    on='Document',
    how='left'
)

merged_df = merged_df.drop_duplicates(subset=['Document', 'created_utc_'])

if not np.issubdtype(merged_df['created_utc_'].dtype, np.datetime64):
    merged_df['created_utc_'] = pd.to_datetime(merged_df['created_utc_'], errors='coerce')

merged_df['month'] = merged_df['created_utc_'].dt.to_period('M')

def get_sequential_palette_for_sector(sector, n_labels):
    sector_cmaps = {
        'transport': 'Reds',
        'housing': 'Oranges',
        'food': 'Blues'
    }
    cmap_name = sector_cmaps.get(sector, 'Greys')
    cmap = plt.get_cmap(cmap_name)
    colors = [cmap(x) for x in np.linspace(0.25, 1, n_labels)]
    return colors

def plot_temporal_sector_results_combined_vertical(
    merged_df, 
    sector_labels, 
    time_col='month',
    save_dir='paper4figs'
):
    """
    Plots temporal trends of label raw counts for all sectors as high-aesthetic stacked area charts in a single figure (one column, three rows).
    Uses a 3-month rolling average for smoothing (centered).
    Font size for x/y/labels/titles is increased by 10% over previous.
    All subplots share the same x-axis range (start and end year/month), so all plots are aligned in time.
    Each subplot has its own y-axis max (not shared).
    Only the bottom plot has an x-axis label; all have y-axis labels.
    Annotation arrows have high transparency (alpha=0.2).
    Curves inside each sector's plot are ordered so that the lowest volume subsector is at the bottom, then next smallest, with biggest on top.
    For each sector, an inset axis is added on the left showing the fraction (percentage) for each subsector (normalized at each time point by the sum of all label counts at that time, which can be >100% due to multi-labels per comment).
    """
    sector_intervention = {
        'transport': 'EVs',
        'housing': 'Solar',
        'food': 'Vegetarianism'
    }

    # Font size settings (10% larger than before)
    BASE_FONT_SIZE = 15
    SMALLER_FONT_SIZE = BASE_FONT_SIZE * 0.7

    os.makedirs(save_dir, exist_ok=True)
    n_sectors = len(sector_labels)
    fig, axes = plt.subplots(n_sectors, 1, figsize=(12, 14), dpi=300, sharex=True, sharey=False)
    if n_sectors == 1:
        axes = [axes]
    elif n_sectors == 0:
        print("No sectors to plot.")
        return

    # --- Find the global min and max month across all sectors ---
    all_months = merged_df['month'].dropna().sort_values().unique()
    if len(all_months) == 0:
        print("No months found in merged_df.")
        return

    # Set the minimum month to 2012-01 if available
    min_month = all_months[0]
    max_month = all_months[-1]
    min_plot_month = pd.Period('2012-01', freq='M')
    # If the data starts after 2012-01, use the data's min month; otherwise, force 2012-01
    if min_month > min_plot_month:
        min_month_for_range = min_month
    else:
        min_month_for_range = min_plot_month

    # Create a full range of months from min_month_for_range to max_month (inclusive)
    all_months_range = pd.period_range(start=min_month_for_range, end=max_month, freq='M')
    months_str = [str(m) for m in all_months_range]

    for idx, (ax, (sector, labels)) in enumerate(zip(axes, sector_labels.items())):
        sector_df = merged_df[merged_df['sector'] == sector].copy()
        if sector_df.empty:
            print(f"No data for sector: {sector}")
            continue

        # --- Compute total volume for each label (for ordering) ---
        label_total_counts = {}
        for label in labels:
            colname = f"{sector}_{label}"
            if colname in sector_df.columns:
                label_total_counts[label] = sector_df[colname].sum()
            else:
                label_total_counts[label] = 0

        # Order labels by total volume (ascending: smallest at bottom, largest at top)
        ordered_labels = sorted(labels, key=lambda l: label_total_counts[l])

        # --- Compute rolling monthly counts for each label in the new order ---
        label_time_counts = {}
        for label in ordered_labels:
            colname = f"{sector}_{label}"
            if colname in sector_df.columns:
                # Group by month and sum, then rolling average (window=3, center=True)
                monthly_counts = sector_df.groupby(time_col)[colname].sum().reindex(all_months_range, fill_value=0)
                rolling_counts = monthly_counts.rolling(window=3, center=True, min_periods=1).mean()
                label_time_counts[label] = rolling_counts
            else:
                label_time_counts[label] = pd.Series(dtype=float, index=all_months_range)

        # Use the global all_months_range for all sectors
        count_matrix = []
        for label in ordered_labels:
            counts = label_time_counts[label].reindex(all_months_range, fill_value=0)
            count_matrix.append(counts.values)
        count_matrix = np.vstack(count_matrix)  # shape: (n_labels, n_months)

        n_labels = len(ordered_labels)
        colors = get_sequential_palette_for_sector(sector, n_labels)

        # Get total number of comments for this sector
        if isinstance(comments_by_sector, dict):
            sector_total_comments = len(comments_by_sector[sector])
        elif hasattr(comments_by_sector, 'loc'):
            sector_total_comments = comments_by_sector.loc[comments_by_sector['sector'] == sector].shape[0]
        else:
            raise ValueError("comments_by_sector must be a dict or DataFrame with sector info.")

        polys = ax.stackplot(
            np.arange(len(months_str)), count_matrix, 
            labels=[label.lower() for label in ordered_labels], 
            colors=colors, 
            alpha=1.0,
            edgecolor='white', 
            linewidth=0.7
         )

        ax.set_facecolor('white')
        ax.grid(axis='y', color='#e5e5e5', linestyle='--', linewidth=1.2, alpha=0.7, zorder=0)

        intervention = sector_intervention.get(sector, "")
        if intervention:
            sector_title = f"{sector.capitalize()} ({intervention})"
        else:
            sector_title = sector.capitalize()
        ax.set_title(
            sector_title,
            fontsize=BASE_FONT_SIZE,
            pad=18,
            loc='left',
            x=0.01,
            y=.8,
            fontweight='bold',
            color='#222222',
            bbox=dict(facecolor='white', edgecolor='#bbbbbb', boxstyle='round,pad=0.18', alpha=0.95)
        )
        # Only set x-axis label for the bottom plot
        if idx == n_sectors - 1:
            ax.set_xlabel("year", fontsize=BASE_FONT_SIZE, labelpad=10, fontweight='semibold')
        # Set y-axis label for all plots, add (rolling) to label
        ax.set_ylabel("monthly counts (rolling)", fontsize=BASE_FONT_SIZE, labelpad=10, fontweight='semibold')
        ax.tick_params(axis='x', labelsize=BASE_FONT_SIZE, rotation=0, length=0)
        ax.tick_params(axis='y', labelsize=BASE_FONT_SIZE, length=0)
        for spine in ax.spines.values():
            spine.set_visible(False)

        # --- Annotate the last value for each label, using adjustText to avoid overlap, with arrows (no head) pointing to original placement ---
        annotation_texts = []
        # No arrows: do not define arrowprops, do not use in adjust_text, do not draw arrows
        for i, label in enumerate(ordered_labels):
            if i >= count_matrix.shape[0]:
                print(f"Warning: Not enough rows in count_matrix for label '{label}' in sector '{sector}'. Skipping annotation.")
                continue
            y_curve = count_matrix[i]
            valid_indices = np.where(~np.isnan(y_curve))[0]
            if len(valid_indices) == 0:
                continue
            last_idx = valid_indices[-1]
            x = last_idx
            y_sum = np.sum(count_matrix[:i+1, x])
            # For annotation, use the sum of the original (not rolling) counts for cumulative count
            colname = f"{sector}_{label}"
            if colname in sector_df.columns:
                orig_monthly_counts = sector_df.groupby(time_col)[colname].sum().reindex(all_months_range, fill_value=0)
                cumulative_count = int(np.nansum(orig_monthly_counts.values))
            else:
                cumulative_count = 0
            if sector_total_comments > 0:
                pct = 100.0 * cumulative_count / sector_total_comments
            else:
                pct = 0.0
            annotation_str = f"{label.replace('_', ' ').capitalize()}\n n={cumulative_count:,} ({pct:.1f}%)"
            annotation = ax.text(
                x, y_sum,
                annotation_str,
                va='center', ha='left',
                fontsize=BASE_FONT_SIZE * 0.7,
                color=colors[i],
                fontweight='bold',
                alpha=1.0,
                bbox=dict(facecolor='white', edgecolor='white', alpha=0.7, pad=0.0)
            )
            annotation_texts.append(annotation)
            annotation._original_xy = (x, y_sum)

        adjust_text(
            annotation_texts,
            ax=ax,
            expand_text=(1.1, 1.2),
            expand_points=(1.2, 1.4),
            # No arrowprops argument
            only_move={'points': 'y', 'text': 'y'},
            autoalign='y',
            force_text=0.5,
            force_points=0.5,
            lim=200,
            precision=0.01
        )
        # Shift all annotation texts right by a fixed amount, and left-align all texts to the same x position
        renderer = ax.figure.canvas.get_renderer()
        trans, inv = ax.transData, ax.transData.inverted()
        # Choose a fixed x position in data coordinates to align all texts (e.g., furthest right + offset)
        # Find the maximum x among all annotation positions
        max_x = max(txt.get_position()[0] for txt in annotation_texts)
        # Shift all texts to a fixed x position to the right of max_x (e.g., by 1.5 units)
        fixed_x = max_x + 15.5
        for txt in annotation_texts:
            _, y = txt.get_position()
            txt.set_position((fixed_x, y))
            txt.set_ha('left')  # Ensure left alignment

        for txt in annotation_texts:
            if hasattr(txt, '_original_xy'):
                txt.set_bbox(dict(facecolor='white', edgecolor='none', alpha=1.0, pad=0.0))

        # --- Custom x-axis: show only years, in gaps of 2 or 3, and ensure x ticks are equally spaced ---
        months_years = [int(str(m)[:4]) for m in all_months_range]
        unique_years = sorted(set(months_years))
        if len(unique_years) > 10:
            step = 3
        else:
            step = 2
        year_ticks = unique_years[::step]
        year_tick_indices = []
        year_tick_labels = []
        for y in year_ticks:
            try:
                idx = months_years.index(y)
                year_tick_indices.append(idx)
                year_tick_labels.append(str(y))
            except ValueError:
                continue
        ax.set_xticks(np.arange(len(months_str)))
        xtick_labels = []
        used_years = set()
        for i, m in enumerate(all_months_range):
            year = int(str(m)[:4])
            if year in year_ticks and months_years[i] == year and year not in used_years:
                xtick_labels.append(str(year))
                used_years.add(year)
            else:
                xtick_labels.append("")
        ax.set_xticklabels(xtick_labels)
        ax.tick_params(axis='x', labelsize=BASE_FONT_SIZE, rotation=0, length=0)

        # Set individual y-axis max for each plot (do NOT share y max)
        y_max = np.nanmax(np.sum(count_matrix, axis=0))
        ax.set_ylim(0, y_max * 1.08 if y_max > 0 else 1)

        # ----------- INSET: Fractional (percentage) stacked area plot -----------
        # For each time point, sum all label counts (can be > number of comments due to multi-labels)
        # Then, for each label, compute fraction of total label counts at that time point
        # Use same color order as main plot, no legend, no axis labels, y-axis 0-1.2 (can sum >1)
        # Place inset on left side of main axis

        # --- Compute cumulative sum of label counts up to each month ---
        # For each label, compute cumulative sum over time
        cumulative_label_counts = np.cumsum(count_matrix, axis=1)  # shape: (n_labels, n_months)
        # For all labels, compute total cumulative sum at each time point
        cumulative_total_counts = np.sum(cumulative_label_counts, axis=0)  # shape: (n_months,)

        # To avoid division by zero, set cumulative_total_counts to 1 where zero (temporarily)
        cumulative_total_counts_safe = np.where(cumulative_total_counts == 0, 1, cumulative_total_counts)
        # For each label, compute fraction of cumulative total at each time point
        cumulative_fraction_matrix = cumulative_label_counts / cumulative_total_counts_safe  # shape: (n_labels, n_months)
        # Set columns (months) where cumulative_total_counts == 0 to all zeros (no data)
        zero_cum_months = (cumulative_total_counts == 0)
        cumulative_fraction_matrix[:, zero_cum_months] = 0.0

        # --- Smooth the cumulative fraction matrix with the same rolling window (3, center=True) ---
        smoothed_cumulative_fraction_matrix = np.zeros_like(cumulative_fraction_matrix)
        for i in range(cumulative_fraction_matrix.shape[0]):
            s = pd.Series(cumulative_fraction_matrix[i])
            smoothed_cumulative_fraction_matrix[i] = s.rolling(window=3, center=True, min_periods=1).mean().values

        # Renormalize so that at each time point, sum of all label fractions is 1 (or 0 if no data)
        col_sums_cum = smoothed_cumulative_fraction_matrix.sum(axis=0)
        col_sums_cum_safe = np.where(col_sums_cum == 0, 1, col_sums_cum)
        fraction_matrix_cum = smoothed_cumulative_fraction_matrix / col_sums_cum_safe
        # Set columns where col_sums_cum==0 to all zeros (no data)
        fraction_matrix_cum[:, col_sums_cum == 0] = 0.0

        # --- Inset axes: place on left, occupying about 52% width, 60% height, left-aligned ---
        inset_width = 0.32
        inset_height = 0.40
        inset_left = 0.1
        inset_bottom = 0.21
        ax_inset = ax.inset_axes([inset_left, inset_bottom, inset_width, inset_height])

        # Plot stacked area of cumulative fractions
        ax_inset.stackplot(
            np.arange(len(months_str)), fraction_matrix_cum,
            colors=colors,
            alpha=1.0,
            edgecolor='white',
            linewidth=0.7
        )
        ax_inset.set_ylim(0, 1)
        ax_inset.set_xlim(0, len(months_str)-1)
        ax_inset.set_facecolor('white')
        # Add 4 x ticks: 2012, 2015, 2018, 2021
        xtick_years = [2012, 2015, 2018, 2021]
        xtick_indices = []
        xtick_labels = []
        for year in xtick_years:
            idx = next((i for i, m in enumerate(months_str) if m.startswith(str(year))), None)
            if idx is not None:
                xtick_indices.append(idx)
                xtick_labels.append(str(year))
        ax_inset.set_xticks(xtick_indices)
        ax_inset.set_xticklabels(xtick_labels, fontsize=BASE_FONT_SIZE*0.7, color='#888888')
        # Add y ticks at 0, 25, 50, 75, 100 percent
        ytick_vals = [0, 0.25, 0.5, 0.75, 1.0]
        ytick_labels = ['0', '25', '50', '75', '100']
        ax_inset.set_yticks(ytick_vals)
        ax_inset.set_yticklabels(ytick_labels, fontsize=BASE_FONT_SIZE*0.7, color='#888888')
        for spine in ax_inset.spines.values():
            spine.set_visible(False)
        for spine in ['left', 'bottom', 'top', 'right']:
            ax_inset.spines[spine].set_visible(True)
            ax_inset.spines[spine].set_color('#bbbbbb')
            ax_inset.spines[spine].set_linewidth(1.0)
        ax_inset.text(0.6, .99, "Cumulative %", fontsize=BASE_FONT_SIZE*0.85, color='k', ha='left', va='bottom', transform=ax_inset.transAxes, alpha=1)

    plt.tight_layout(rect=[0, 0, 1, 1])
    plt.savefig(os.path.join(save_dir, f"paper4_fig2_temporal_allsectors_stackedarea_vertical.pdf"), bbox_inches='tight', dpi=300)
    plt.savefig(os.path.join(save_dir, f"paper4_fig2_temporal_allsectors_stackedarea_vertical.png"), bbox_inches='tight', dpi=300)
    plt.close()
    print("Plotted and saved high-aesthetic stacked area temporal trend for all sectors in a single vertical figure (3-month rolling average), with left insets showing per-label fraction.")

# Call the function
plot_temporal_sector_results_combined_vertical(merged_df, sector_labels)


Plotted and saved high-aesthetic stacked area temporal trend for all sectors in a single vertical figure (3-month rolling average), with left insets showing per-label fraction.


# Pro anti GPT training, test data

In [97]:


# Reload the module and set up for train/val/test splits
from importlib import reload
import paper4_pro_anti_GPT_classifier
reload(paper4_pro_anti_GPT_classifier)
from paper4_pro_anti_GPT_classifier import classify_comments_pro_anti, main_pro_anti

import json

# User option: choose starting point for range
START_IDX = 5000  # <-- Set this to your desired starting index (e.g., 0, 1000, etc.)

# Define split sizes and ranges (150 per batch)
split_sizes = {
    "train": 600,      # 4 batches of 150
    "validation": 150, # 1 batch of 150
    "test": 150        # 1 batch of 150
}
batch_size = 150

# For each sector, get the comments as a list
comments_by_sector_lists = {
    sector: list(comments)
    for sector, comments in comments_by_sector.items()
}

# Helper to get batch indices for each split, with custom start
def get_split_indices(split, split_sizes, batch_size, start_idx):
    if split == "train":
        # 4 batches: start_idx to start_idx+600
        return [(start_idx + i * batch_size, start_idx + (i + 1) * batch_size) for i in range(split_sizes["train"] // batch_size)]
    elif split == "validation":
        # Next batch: start_idx+600 to start_idx+750
        return [(start_idx + split_sizes["train"], start_idx + split_sizes["train"] + batch_size)]
    elif split == "test":
        # Next batch: start_idx+750 to start_idx+900
        return [(start_idx + split_sizes["train"] + batch_size, start_idx + split_sizes["train"] + 2 * batch_size)]
    else:
        raise ValueError("Unknown split")

splits = ["train", "validation", "test"]

for split in splits:
    split_indices = get_split_indices(split, split_sizes, batch_size, START_IDX)
    # For each batch in the split, process and accumulate results
    all_results = {}
    all_results_df = []
    for batch_num, (start, end) in enumerate(split_indices, start=1):
        # Sample comments for this batch for each sector
        sampled_comments = {
            sector: comments[start:end] if len(comments) > end else comments[start:]
            for sector, comments in comments_by_sector_lists.items()
        }
        # Run pro/anti classifier for all sectors
        results, results_df = main_pro_anti(
            comments_by_sector=sampled_comments,
            num_runs=1,
            max_comments_per_sector=1000,
            num_agents=3,
            batch_size=1
        )
        # Accumulate results
        all_results[f"batch_{batch_num}"] = results
        all_results_df.append(results_df)

    # Save results with split name in filenames
    results_filename = f"paper4data/{split}_data_by_GPT_results_pro_anti_v2.json"
    results_df_filename = f"paper4data/{split}_data_by_GPT_df_pro_anti_v2.csv"

    with open(results_filename, 'w') as f:
        json.dump(all_results, f, indent=2)
    # Concatenate all DataFrames for this split
    import pandas as pd
    pd.concat(all_results_df, ignore_index=True).to_csv(results_df_filename, index=False)



Processing transport sector with 150 comments...
Processing transport (Electric Vehicles) with 150 comments...
Processing 100 comments for Electric Vehicles with 3 parallel agents, batch size 1, 1 runs per batch
Created 100 batches of 1 comments each


Processing Electric Vehicles batches: 100%|██████████| 100/100 [00:30<00:00,  3.30task/s]



Completed 100/100 tasks
Processed 100 batches with 100 successful runs
✓ Results saved to paper4data/pro_anti_transport_*
DataFrame created with shape: (100, 6)
Comments per topic:
- Electric Vehicles: 100 comments

Sample scores (should be between 0.0 and 1.0):
Electric Vehicles:               pro        anti  pro_and_anti     neither
count  100.000000  100.000000    100.000000  100.000000
mean     0.180000    0.310000      0.020000    0.490000
std      0.386123    0.464823      0.140705    0.502418
min      0.000000    0.000000      0.000000    0.000000
25%      0.000000    0.000000      0.000000    0.000000
50%      0.000000    0.000000      0.000000    0.000000
75%      0.000000    1.000000      0.000000    1.000000
max      1.000000    1.000000      1.000000    1.000000
transport completed in 30.32 seconds

Processing housing sector with 150 comments...
Processing housing (Solar Power) with 150 comments...
Processing 100 comments for Solar Power with 3 parallel agents, batch size

Processing Solar Power batches: 100%|██████████| 100/100 [00:24<00:00,  4.14task/s]



Completed 100/100 tasks
Processed 100 batches with 100 successful runs
✓ Results saved to paper4data/pro_anti_housing_*
DataFrame created with shape: (100, 6)
Comments per topic:
- Solar Power: 100 comments

Sample scores (should be between 0.0 and 1.0):
Solar Power:               pro        anti  pro_and_anti     neither
count  100.000000  100.000000    100.000000  100.000000
mean     0.140000    0.280000      0.020000    0.560000
std      0.348735    0.451261      0.140705    0.498888
min      0.000000    0.000000      0.000000    0.000000
25%      0.000000    0.000000      0.000000    0.000000
50%      0.000000    0.000000      0.000000    1.000000
75%      0.000000    1.000000      0.000000    1.000000
max      1.000000    1.000000      1.000000    1.000000
housing completed in 24.16 seconds

Processing food sector with 150 comments...
Processing food (Vegetarianism/Veganism) with 150 comments...
Processing 100 comments for Vegetarianism/Veganism with 3 parallel agents, batch size

...


Completed 100/100 tasks
Processed 100 batches with 100 successful runs
✓ Results saved to paper4data/pro_anti_transport_*
DataFrame created with shape: (100, 6)
Comments per topic:
- Electric Vehicles: 100 comments

Sample scores (should be between 0.0 and 1.0):
Electric Vehicles:              pro        anti  pro_and_anti     neither
count  100.00000  100.000000    100.000000  100.000000
mean     0.21000    0.350000      0.020000    0.420000
std      0.40936    0.479372      0.140705    0.496045
min      0.00000    0.000000      0.000000    0.000000
25%      0.00000    0.000000      0.000000    0.000000
50%      0.00000    0.000000      0.000000    0.000000
75%      0.00000    1.000000      0.000000    1.000000
max      1.00000    1.000000      1.000000    1.000000
transport completed in 29.18 seconds

Processing housing sector with 150 comments...
Processing housing (Solar Power) with 150 comments...
Processing 100 comments for Solar Power with 3 parallel agents, batch size 1, 1 run

Processing Solar Power batches: 100%|██████████| 100/100 [00:23<00:00,  4.18task/s]



Completed 100/100 tasks
Processed 100 batches with 100 successful runs
✓ Results saved to paper4data/pro_anti_housing_*
DataFrame created with shape: (100, 6)
Comments per topic:
- Solar Power: 100 comments

Sample scores (should be between 0.0 and 1.0):
Solar Power:               pro        anti  pro_and_anti     neither
count  100.000000  100.000000    100.000000  100.000000
mean     0.270000    0.230000      0.030000    0.470000
std      0.446196    0.422953      0.171447    0.501614
min      0.000000    0.000000      0.000000    0.000000
25%      0.000000    0.000000      0.000000    0.000000
50%      0.000000    0.000000      0.000000    0.000000
75%      1.000000    0.000000      0.000000    1.000000
max      1.000000    1.000000      1.000000    1.000000
housing completed in 23.92 seconds

Processing food sector with 150 comments...
Processing food (Vegetarianism/Veganism) with 150 comments...
Processing 100 comments for Vegetarianism/Veganism with 3 parallel agents, batch size

Processing Vegetarianism/Veganism batches: 100%|██████████| 100/100 [00:31<00:00,  3.18task/s]


Completed 100/100 tasks
Processed 100 batches with 100 successful runs
✓ Results saved to paper4data/pro_anti_food_*
DataFrame created with shape: (100, 6)
Comments per topic:
- Vegetarianism/Veganism: 100 comments

Sample scores (should be between 0.0 and 1.0):
Vegetarianism/Veganism:               pro        anti  pro_and_anti     neither
count  100.000000  100.000000     100.00000  100.000000
mean     0.390000    0.170000       0.08000    0.360000
std      0.490207    0.377525       0.27266    0.482418
min      0.000000    0.000000       0.00000    0.000000
25%      0.000000    0.000000       0.00000    0.000000
50%      0.000000    0.000000       0.00000    0.000000
75%      1.000000    0.000000       0.00000    1.000000
max      1.000000    1.000000       1.00000    1.000000
food completed in 31.42 seconds

Total processing time: 84.52 seconds
✓ Results saved to paper4data/pro_anti_all_sectors_*
DataFrame created with shape: (300, 6)
Comments per topic:
- Electric Vehicles: 100 c

In [110]:
import pandas as pd

# Only compare between these two CSVs
csv1 = "paper4data/train_data_by_GPT_df_pro_anti_2.csv"
csv2 = "paper4data/train_data_by_GPT_df_pro_anti_2v2.csv"

label_cols = ["pro", "anti", "pro_and_anti", "neither"]

def get_labels_for_row(row, label_cols):
    """
    For a row, return a set of label columns (from label_cols) that are 1/True.
    """
    return set(label for label in label_cols if label in row and (row[label] == 1 or row[label] is True or row[label] == True))

try:
    df1 = pd.read_csv(csv1)
    df2 = pd.read_csv(csv2)

    # Ensure all required columns exist
    for col in label_cols:
        if col not in df1.columns or col not in df2.columns:
            raise ValueError(f"Column '{col}' missing in one of the files: {csv1}, {csv2}")
    if 'comment' not in df1.columns or 'comment' not in df2.columns:
        raise ValueError("Both files must have a 'comment' column.")

    # Ensure all comments match
    comments1 = set(df1['comment'])
    comments2 = set(df2['comment'])
    all_comments_match = comments1 == comments2
    print(f"All comments match: {all_comments_match}")
    if not all_comments_match:
        only_in_1 = comments1 - comments2
        only_in_2 = comments2 - comments1
        print(f"Comments only in {csv1}: {len(only_in_1)}")
        print(f"Comments only in {csv2}: {len(only_in_2)}")
    n_comments_1 = len(comments1)
    n_comments_2 = len(comments2)
    print(f"Number of unique comments in {csv1}: {n_comments_1}")
    print(f"Number of unique comments in {csv2}: {n_comments_2}")

    # Merge on comment (inner join to only compare comments present in both)
    merged = pd.merge(
        df1[['comment'] + label_cols],
        df2[['comment'] + label_cols],
        on='comment',
        how='inner'
    )
    n_matched_comments = len(merged)
    print(f"Number of matched comments (intersection): {n_matched_comments}")

    # The percentage of matched comments should be relative to the union, not the max, to avoid >100%
    n_union_comments = len(comments1 | comments2)
    percent_matched_union = (n_matched_comments / n_union_comments) * 100 if n_union_comments > 0 else 0.0
    print(f"Percentage of comments matched (intersection/union): {percent_matched_union:.2f}%")

    # For reference, also print the old calculation (intersection/max) and explain why it can be >100%
    percent_matched_max = (n_matched_comments / max(n_comments_1, n_comments_2)) * 100 if max(n_comments_1, n_comments_2) > 0 else 0.0
    if percent_matched_max > 100:
        print(f"WARNING: Percentage of comments matched (intersection/max): {percent_matched_max:.2f}% (>100% means there are duplicate comments in at least one file)")
        print("This can happen if there are duplicate 'comment' values in one or both files. The intersection (number of rows in merged) can exceed the number of unique comments in either file if duplicates exist.")
        print("Check for duplicates in each file:")
        n_dupes_1 = df1['comment'].duplicated().sum()
        n_dupes_2 = df2['comment'].duplicated().sum()
        print(f"Number of duplicate comments in {csv1}: {n_dupes_1}")
        print(f"Number of duplicate comments in {csv2}: {n_dupes_2}")
    else:
        print(f"Percentage of comments matched (intersection/max): {percent_matched_max:.2f}%")

    # For each matched comment, get the set of labels in each file
    def get_labels_for_row1(row):
        return set(label for label in label_cols if row[label + '_x'] == 1 or row[label + '_x'] is True or row[label + '_x'] == True)
    def get_labels_for_row2(row):
        return set(label for label in label_cols if row[label + '_y'] == 1 or row[label + '_y'] is True or row[label + '_y'] == True)

    # After merge, columns from df1 have _x, from df2 have _y
    merged['labels_1'] = merged.apply(get_labels_for_row1, axis=1)
    merged['labels_2'] = merged.apply(get_labels_for_row2, axis=1)

    # Calculate overlap percentage (labels_1 == labels_2)
    merged['label_overlap'] = merged.apply(lambda row: row['labels_1'] == row['labels_2'], axis=1)
    n_overlap = merged['label_overlap'].sum()
    percent_overlap = (n_overlap / n_matched_comments) * 100 if n_matched_comments > 0 else 0.0
    print(f"Label set exact overlap: {percent_overlap:.2f}% ({n_overlap}/{n_matched_comments})")

    # Show a sample of the merged DataFrame
    print(merged[['comment', 'labels_1', 'labels_2', 'label_overlap']].head(10))

except Exception as e:
    print(f"Error comparing {csv1} and {csv2}: {e}")


All comments match: True
Number of unique comments in paper4data/train_data_by_GPT_df_pro_anti_2.csv: 1180
Number of unique comments in paper4data/train_data_by_GPT_df_pro_anti_2v2.csv: 1180
Number of matched comments (intersection): 1240
Percentage of comments matched (intersection/union): 105.08%
This can happen if there are duplicate 'comment' values in one or both files. The intersection (number of rows in merged) can exceed the number of unique comments in either file if duplicates exist.
Check for duplicates in each file:
Number of duplicate comments in paper4data/train_data_by_GPT_df_pro_anti_2.csv: 20
Number of duplicate comments in paper4data/train_data_by_GPT_df_pro_anti_2v2.csv: 20
Label set exact overlap: 96.05% (1191/1240)
                                             comment   labels_1   labels_2  \
0                Falsely Powering Up the EV Industry     {anti}     {anti}   
1  EVs seem to be there mainly to assuage upper a...     {anti}     {anti}   
2  Do you really cla

# finteuning roberta for pro and anti labellling

In [92]:
# Run the training
# !python paper4_BERT_finetuning_pro_anti.py


In [ ]:

# Or import and use functions
from paper4_BERT_finetuning_pro_anti import debug_training_pro_anti_with_evaluation
debug_training_pro_anti_with_evaluation()

## pro anti BERT predictions

In [8]:

"""
Get pro-anti predictions for comments using fine-tuned BERT models
Similar to get_model_predictions but for pro-anti classification
"""

import os
import glob
import torch
import pandas as pd
from tqdm import tqdm
import json

# Import functions from pro-anti training and evaluation files
from paper4_BERT_finetuning_pro_anti import (
    ProAntiClassifier,
    get_tokenizer,
    predict_pro_anti_scores,
    topic_to_sector
)

from paper4_BERT_finetuning_pro_anti_eval import (
    apply_optimal_thresholds
)

def load_trained_pro_anti_model(model_dir, device):
    """
    Load a trained pro-anti model from directory
    
    Args:
        model_dir: Path to model directory
        device: torch device
    
    Returns:
        model, tokenizer, label_names, optimal_thresholds
    """
    try:
        # Load model checkpoint
        checkpoint_path = os.path.join(model_dir, 'best_model', 'model.pt')
        if not os.path.exists(checkpoint_path):
            print(f"Best model not found at {checkpoint_path}")
            return None, None, None, None
        
        checkpoint = torch.load(checkpoint_path, map_location=device, weights_only=False)
        model_name = checkpoint.get('model_name', 'roberta-base')
        label_names = checkpoint['label_names']
        optimal_thresholds = checkpoint.get('optimal_thresholds', {})
        
        print(f"Loading {model_name} model with labels: {label_names}")
        
        # Initialize model
        model = ProAntiClassifier(model_name, len(label_names))
        model.load_state_dict(checkpoint['model_state_dict'])
        model = model.to(device)
        model.eval()
        
        # Load tokenizer
        tokenizer = get_tokenizer(model_name)
        tokenizer_path = os.path.join(model_dir, 'best_model')
        if os.path.exists(tokenizer_path):
            tokenizer = tokenizer.from_pretrained(tokenizer_path)
        
        return model, tokenizer, label_names, optimal_thresholds
        
    except Exception as e:
        print(f"Error loading model from {model_dir}: {e}")
        return None, None, None, None

def get_pro_anti_predictions(comments_by_sector, 
                           save_dir="paper4data/sectorwise_pro_anti_classifications",
                           thresholded_save_dir="paper4data/sectorwise_pro_anti_classifications_thresholded"):
    """
    Get pro-anti predictions for comments using fine-tuned BERT models
    
    Args:
        comments_by_sector: Dict of {sector: [comments]}
        save_dir: Directory to save raw predictions
        thresholded_save_dir: Directory to save thresholded predictions
    
    Returns:
        results_df, sector_labels
    """
    device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
    print(f"Using device: {device}")
    
    # Find and load all trained pro-anti models
    model_dirs = glob.glob("models/pro_anti_*_roberta_base")
    if not model_dirs:
        # Fallback to DistilBERT models if RoBERTa not found
        model_dirs = glob.glob("models/pro_anti_*_distilbert_base_uncased")
    
    if not model_dirs:
        print("No trained pro-anti models found!")
        print("Expected directories: models/pro_anti_*_roberta_base or models/pro_anti_*_distilbert_base_uncased")
        return None, None
    
    print(f"Found {len(model_dirs)} pro-anti model directories: {model_dirs}")
    
    # Load all sector models
    all_sector_models = {}
    optimal_thresholds = {}
    sector_labels = {}
    
    for model_dir in model_dirs:
        # Extract sector from directory name (e.g., pro_anti_food_roberta_base -> food)
        sector = model_dir.split('_')[2]  # pro_anti_SECTOR_model
        
        print(f"Loading {sector.upper()} pro-anti model...")
        
        # Load model
        model, tokenizer, label_names, sector_optimal_thresholds = load_trained_pro_anti_model(model_dir, device)
        
        if model is None:
            print(f"Failed to load {sector} pro-anti model")
            continue
            
        all_sector_models[sector] = {
            'model': model,
            'tokenizer': tokenizer,
            'label_names': label_names
        }
        
        optimal_thresholds[sector] = sector_optimal_thresholds
        sector_labels[sector] = label_names
        
        print(f"✓ Loaded {sector} model with labels: {label_names}")
        print(f"  Optimal thresholds: {sector_optimal_thresholds}")
    
    if not all_sector_models:
        print("No pro-anti models loaded successfully!")
        return None, None
        
    # Ensure output directories exist
    os.makedirs(save_dir, exist_ok=True)
    os.makedirs(thresholded_save_dir, exist_ok=True)
    
    # Store all results for return
    all_results = []
    all_thresholded_results = []
    
    # Process each sector
    for sector, comments in comments_by_sector.items():
        if sector not in all_sector_models:
            print(f"Skipping {sector} - no model available")
            continue
            
        print(f"\nProcessing {sector} comments for pro-anti classification...")
        
        model = all_sector_models[sector]['model']
        tokenizer = all_sector_models[sector]['tokenizer']
        label_names = all_sector_models[sector]['label_names']
        
        # Get predictions for each comment
        sector_results = []
        sector_thresholded_results = []
        
        for text in tqdm(comments, desc=f"Processing {sector}"):
            # Get predictions
            prediction_results = predict_pro_anti_scores(text, model, tokenizer, label_names, device)
            scores = [res['score'] for res in prediction_results]
            
            # Create result dict
            result = {'sector': sector, 'text': text}
            
            # Add scores for each label (pro, anti, neither)
            for label, score in zip(label_names, scores):
                result[f"{sector}_{label}"] = score
                
            sector_results.append(result)
        
        # Save raw results for this sector
        sector_df = pd.DataFrame(sector_results)
        sector_csv_path = os.path.join(save_dir, f"{sector}_pro_anti_classifications.csv")
        sector_df.to_csv(sector_csv_path, index=False)
        print(f"Saved {sector} pro-anti classifications to {sector_csv_path}")
        all_results.extend(sector_results)
        
        # Now create thresholded version for this sector
        thresholds = optimal_thresholds.get(sector, {})
        print(f"Applying thresholds for {sector}: {thresholds}")
        
        for row in sector_results:
            thresholded_row = {'sector': sector, 'text': row['text']}
            
            # Apply optimal thresholds to get binary predictions
            scores = [row[f"{sector}_{label}"] for label in label_names]
            
            if thresholds:
                # Use optimal thresholds if available
                pred_binary = apply_optimal_thresholds(scores, {sector: thresholds}, sector, label_names)
            else:
                # Default threshold of 0.5
                pred_binary = [1 if score >= 0.5 else 0 for score in scores]
            
            # Add binary predictions
            for label, pred in zip(label_names, pred_binary):
                thresholded_row[f"{sector}_{label}"] = pred
            
            # Add predicted label (highest score after thresholding)
            if sum(pred_binary) > 0:
                predicted_label_idx = pred_binary.index(1)  # Get first label with 1
                predicted_label = label_names[predicted_label_idx]
            else:
                predicted_label = 'neither'  # Default if no labels above threshold
            
            thresholded_row[f"{sector}_predicted_label"] = predicted_label
            
            sector_thresholded_results.append(thresholded_row)
        
        # Save thresholded results for this sector
        thresholded_sector_df = pd.DataFrame(sector_thresholded_results)
        thresholded_sector_csv_path = os.path.join(thresholded_save_dir, f"{sector}_pro_anti_classifications_thresholded.csv")
        thresholded_sector_df.to_csv(thresholded_sector_csv_path, index=False)
        print(f"Saved thresholded {sector} pro-anti classifications to {thresholded_sector_csv_path}")
        all_thresholded_results.extend(sector_thresholded_results)
    
    # Create combined results dataframes
    results_df = pd.DataFrame(all_results)
    thresholded_df = pd.DataFrame(all_thresholded_results)
    
    # Save combined results
    combined_csv_path = os.path.join(save_dir, "all_sectors_pro_anti_classifications.csv")
    results_df.to_csv(combined_csv_path, index=False)
    print(f"Saved combined raw pro-anti classifications to {combined_csv_path}")
    
    combined_thresholded_csv_path = os.path.join(thresholded_save_dir, "all_sectors_pro_anti_classifications_thresholded.csv")
    thresholded_df.to_csv(combined_thresholded_csv_path, index=False)
    print(f"Saved combined thresholded pro-anti classifications to {combined_thresholded_csv_path}")
    
    # Print summary statistics
    print(f"\n{'='*60}")
    print("PRO-ANTI CLASSIFICATION SUMMARY")
    print(f"{'='*60}")
    
    for sector in all_sector_models.keys():
        if sector in thresholded_df['sector'].values:
            sector_data = thresholded_df[thresholded_df['sector'] == sector]
            
            # Count predictions for each label
            label_counts = {}
            for label in sector_labels[sector]:
                col = f"{sector}_{label}"
                if col in sector_data.columns:
                    label_counts[label] = sector_data[col].sum()
            
            print(f"{sector.upper()}:")
            for label, count in label_counts.items():
                percentage = (count / len(sector_data)) * 100
                print(f"  {label}: {count} ({percentage:.1f}%)")
            print()
    
    return results_df, sector_labels

def main():
    """Main function to run pro-anti predictions"""
    print("="*80)
    print("PRO-ANTI CLASSIFICATION USING FINE-TUNED BERT MODELS")
    print("="*80)
    
    # Example usage - you would replace this with your actual comments_by_sector data
    # comments_by_sector = {
    #     'food': ['comment1', 'comment2', ...],
    #     'housing': ['comment1', 'comment2', ...],
    #     'transport': ['comment1', 'comment2', ...]
    # }
    
    print("Please provide your comments_by_sector data and call get_pro_anti_predictions()")
    print("Example:")
    print("results_df, sector_labels = get_pro_anti_predictions(comments_by_sector)")
    
    print(f"\nOutput directories:")
    print(f"  Raw predictions: paper4data/sectorwise_pro_anti_classifications/")
    print(f"  Thresholded predictions: paper4data/sectorwise_pro_anti_classifications_thresholded/")
    
    print(f"\nExpected model directories:")
    print(f"  models/pro_anti_*_roberta_base")
    print(f"  models/pro_anti_*_distilbert_base_uncased")

if __name__ == "__main__":
    main()

c:\Users\chowdhary\Desktop\Redit project 4.09.2024\sbert_env\Lib\site-packages\tqdm\auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


PRO-ANTI CLASSIFICATION USING FINE-TUNED BERT MODELS
Please provide your comments_by_sector data and call get_pro_anti_predictions()
Example:
results_df, sector_labels = get_pro_anti_predictions(comments_by_sector)

Output directories:
  Raw predictions: paper4data/sectorwise_pro_anti_classifications/
  Thresholded predictions: paper4data/sectorwise_pro_anti_classifications_thresholded/

Expected model directories:
  models/pro_anti_*_roberta_base
  models/pro_anti_*_distilbert_base_uncased


In [14]:
# from paper4_predictionss_by_finetunedBERT import get_pro_anti_predictions

# Run predictions for all comments in each sector
results_df, sector_labels = get_pro_anti_predictions(comments_by_sector)

Using device: cpu
Found 3 pro-anti model directories: ['models\\pro_anti_food_roberta_base', 'models\\pro_anti_housing_roberta_base', 'models\\pro_anti_transport_roberta_base']
Loading FOOD pro-anti model...
Loading roberta-base model with labels: ['pro', 'anti', 'neither']


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Loaded food model with labels: ['pro', 'anti', 'neither']
  Optimal thresholds: {'pro': 0.35000000000000003, 'anti': 0.5, 'neither': 0.6000000000000001}
Loading HOUSING pro-anti model...
Loading roberta-base model with labels: ['pro', 'anti', 'neither']


Some weights of RobertaModel were not initialized from the model checkpoint at roberta-base and are newly initialized: ['pooler.dense.bias', 'pooler.dense.weight']
You should probably TRAIN this model on a down-stream task to be able to use it for predictions and inference.


✓ Loaded housing model with labels: ['pro', 'anti', 'neither']
  Optimal thresholds: {'pro': 0.55, 'anti': 0.55, 'neither': 0.45}
Loading TRANSPORT pro-anti model...
Loading roberta-base model with labels: ['pro', 'anti', 'neither']


...

Saved transport pro-anti classifications to paper4data/sectorwise_pro_anti_classifications\transport_pro_anti_classifications.csv
Applying thresholds for transport: {'pro': 0.5, 'anti': 0.5, 'neither': 0.5}
Saved thresholded transport pro-anti classifications to paper4data/sectorwise_pro_anti_classifications_thresholded\transport_pro_anti_classifications_thresholded.csv

Processing housing comments for pro-anti classification...


Processing housing: 100%|██████████| 9388/9388 [36:16<00:00,  4.31it/s]  


Saved housing pro-anti classifications to paper4data/sectorwise_pro_anti_classifications\housing_pro_anti_classifications.csv
Applying thresholds for housing: {'pro': 0.55, 'anti': 0.55, 'neither': 0.45}
Saved thresholded housing pro-anti classifications to paper4data/sectorwise_pro_anti_classifications_thresholded\housing_pro_anti_classifications_thresholded.csv

Processing food comments for pro-anti classification...


Processing food: 100%|██████████| 8414/8414 [22:27<00:00,  6.25it/s]  


Saved food pro-anti classifications to paper4data/sectorwise_pro_anti_classifications\food_pro_anti_classifications.csv
Applying thresholds for food: {'pro': 0.35000000000000003, 'anti': 0.5, 'neither': 0.6000000000000001}
Saved thresholded food pro-anti classifications to paper4data/sectorwise_pro_anti_classifications_thresholded\food_pro_anti_classifications_thresholded.csv
Saved combined raw pro-anti classifications to paper4data/sectorwise_pro_anti_classifications\all_sectors_pro_anti_classifications.csv
Saved combined thresholded pro-anti classifications to paper4data/sectorwise_pro_anti_classifications_thresholded\all_sectors_pro_anti_classifications_thresholded.csv

PRO-ANTI CLASSIFICATION SUMMARY
FOOD:
  pro: 3840.0 (45.6%)
  anti: 2203.0 (26.2%)
  neither: 2250.0 (26.7%)

HOUSING:
  pro: 2834.0 (30.2%)
  anti: 2922.0 (31.1%)
  neither: 3401.0 (36.2%)

TRANSPORT:
  pro: 3709.0 (24.9%)
  anti: 5573.0 (37.3%)
  neither: 5471.0 (36.7%)



# TESTING THE HUGGINFACE UPLOADS

In [ ]:
import torch, sys, os, tempfile
from transformers import DistilBertTokenizer
from huggingface_hub import snapshot_download

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def print_sorted_label_scores(label_scores):
    # Sort label_scores dict by score descending
    sorted_items = sorted(label_scores.items(), key=lambda x: x[1], reverse=True)
    for label, score in sorted_items:
        print(f"  {label}: {score:.6f}")


# ------------------ TRANSPORT ------------------
transport_model_link = 'sanchow/electric_vehicles-distilbert-classifier'
transport_examples = [
    "Switching to electric cars can cut down on smog and carbon output."
]

print(f"\n{'='*60}")
print("MODEL: TRANSPORT SECTOR")
print(f"{'='*60}")
print(f"Downloading model: {transport_model_link}")
with tempfile.TemporaryDirectory() as temp_dir:
    snapshot_download(
        repo_id=transport_model_link,
        local_dir=temp_dir,
        local_dir_use_symlinks=False
    )
    model_class_path = os.path.join(temp_dir, 'model_class.py')
    if not os.path.exists(model_class_path):
        print(f"model_class.py not found in downloaded files")
        print(f"   Available files: {os.listdir(temp_dir)}")
    else:
        sys.path.insert(0, temp_dir)
        from model_class import MultilabelClassifier
        tokenizer = DistilBertTokenizer.from_pretrained(temp_dir)
        checkpoint = torch.load(os.path.join(temp_dir, 'model.pt'), map_location='cpu', weights_only=False)
        model = MultilabelClassifier(checkpoint['model_name'], len(checkpoint['label_names']))
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        model.eval()
        print("Model loaded successfully")
        print(f"   Labels: {checkpoint['label_names']}")
        print("\nTransport classifier results for transport_examples:\n")
        for i, test_text in enumerate(transport_examples):
            inputs = tokenizer(
                test_text, 
                return_tensors="pt", 
                truncation=True, 
                max_length=512,
                padding=True
            ).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = outputs.cpu().numpy() if isinstance(outputs, (tuple, list)) else outputs.cpu().numpy()
            label_scores = {label: float(score) for label, score in zip(checkpoint['label_names'], predictions[0])}
            print(f"Example {i+1}: '{test_text}'")
            print("Predictions (all label scores, highest first):")
            print_sorted_label_scores(label_scores)
            print("-" * 40)

# ------------------ HOUSING ------------------
housing_model_link = 'sanchow/solar_energy-distilbert-classifier'
housing_examples = [
    "Solar panels can help reduce electricity bills over time.",
    "Disposing of old solar panels is a growing environmental concern.",
    "Relying on imported solar panels could be risky for our economy.",
    "Solar power can make the grid more resilient during peak demand.",
    "Installing solar panels increases the value of my home."
]

print(f"\n{'='*60}")
print("MODEL: HOUSING SECTOR")
print(f"{'='*60}")
print(f"Downloading model: {housing_model_link}")
with tempfile.TemporaryDirectory() as temp_dir:
    snapshot_download(
        repo_id=housing_model_link,
        local_dir=temp_dir,
        local_dir_use_symlinks=False
    )
    model_class_path = os.path.join(temp_dir, 'model_class.py')
    if not os.path.exists(model_class_path):
        print(f"model_class.py not found in downloaded files")
        print(f"   Available files: {os.listdir(temp_dir)}")
    else:
        sys.path.insert(0, temp_dir)
        from model_class import MultilabelClassifier
        tokenizer = DistilBertTokenizer.from_pretrained(temp_dir)
        checkpoint = torch.load(os.path.join(temp_dir, 'model.pt'), map_location='cpu', weights_only=False)
        model = MultilabelClassifier(checkpoint['model_name'], len(checkpoint['label_names']))
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        model.eval()
        print("Model loaded successfully")
        print(f"   Labels: {checkpoint['label_names']}")
        print("\nHousing classifier results for housing_examples:\n")
        for i, test_text in enumerate(housing_examples):
            inputs = tokenizer(
                test_text, 
                return_tensors="pt", 
                truncation=True, 
                max_length=512,
                padding=True
            ).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                predictions = outputs.cpu().numpy() if isinstance(outputs, (tuple, list)) else outputs.cpu().numpy()
            label_scores = {label: float(score) for label, score in zip(checkpoint['label_names'], predictions[0])}
            print(f"Example {i+1}: '{test_text}'")
            print("Predictions (all label scores, highest first):")
            print_sorted_label_scores(label_scores)
            print("-" * 40)

# ------------------ FOOD ------------------
food_model_link = 'sanchow/veganism_and_vegetarianism-distilbert-classifier'
food_examples = [
    "A plant-based diet is better for the environment.",
    "I can't give up cheese, even though I know veganism is healthier.",
    "Lab-grown meat could be the future of sustainable food.",
    "Being vegan is part of my identity and values.",
    "Eating less meat is a personal choice, not a systemic solution."
]

print(f"\n{'='*60}")
print("MODEL: FOOD SECTOR")
print(f"{'='*60}")
print(f"Downloading model: {food_model_link}")
with tempfile.TemporaryDirectory() as temp_dir:
    snapshot_download(
        repo_id=food_model_link,
        local_dir=temp_dir,
        local_dir_use_symlinks=False
    )
    model_class_path = os.path.join(temp_dir, 'model_class.py')
    if not os.path.exists(model_class_path):
        print(f"model_class.py not found in downloaded files")
        print(f"   Available files: {os.listdir(temp_dir)}")
    else:
        sys.path.insert(0, temp_dir)
        from model_class import MultilabelClassifier
        tokenizer = DistilBertTokenizer.from_pretrained(temp_dir)
        checkpoint = torch.load(os.path.join(temp_dir, 'model.pt'), map_location='cpu', weights_only=False)
        model = MultilabelClassifier(checkpoint['model_name'], len(checkpoint['label_names']))
        model.load_state_dict(checkpoint['model_state_dict'])
        model.to(device)
        model.eval()
        print("Model loaded successfully")
        print(f"   Labels: {checkpoint['label_names']}")
        print("\nFood classifier results for food_examples:\n")
        for i, test_text in enumerate(food_examples):
            inputs = tokenizer(
                test_text, 
                return_tensors="pt", 
                truncation=True, 
                max_length=512,
                padding=True
            ).to(device)
            with torch.no_grad():
                outputs = model(**inputs)
                # outputs shape: (batch_size, num_labels)
                predictions = outputs.cpu().numpy() if isinstance(outputs, (tuple, list)) else outputs.cpu().numpy()
            # Print all label scores for this example, sorted highest first
            label_scores = {label: float(score) for label, score in zip(checkpoint['label_names'], predictions[0])}
            print(f"Example {i+1}: '{test_text}'")
            print("Predictions (all label scores, highest first):")
            print_sorted_label_scores(label_scores)
            print("-" * 40)


MODEL: TRANSPORT SECTOR


Fetching 7 files: 100%|██████████| 7/7 [00:08<00:00,  1.26s/it]


Model loaded successfully
   Labels: ['Alternative Modes', 'Charging Infrastructure', 'Environmental Benefit', 'Grid Impact And Energy Mix', 'Mineral Supply Chain', 'Policy And Mandates', 'Purchase Price']

Transport classifier results for transport_examples:

Example 1: 'I usually opt for the subway or cycling rather than using my own vehicle.'
Predictions (all label scores, highest first):
  Alternative Modes: 0.637089
  Environmental Benefit: 0.249878
  Purchase Price: 0.109989
  Charging Infrastructure: 0.098296
  Policy And Mandates: 0.081984
  Mineral Supply Chain: 0.035143
  Grid Impact And Energy Mix: 0.027300
----------------------------------------
Example 2: 'It's hard to find places to plug in my electric car when I'm out.'
Predictions (all label scores, highest first):
  Charging Infrastructure: 0.509047
  Purchase Price: 0.193760
  Alternative Modes: 0.107299
  Environmental Benefit: 0.093271
  Policy And Mandates: 0.050311
  Mineral Supply Chain: 0.021624
  Grid Impact A

Fetching 7 files: 100%|██████████| 7/7 [00:11<00:00,  1.67s/it]


Model loaded successfully
   Labels: ['Decommissioning And Waste', 'Foreign Dependence And Trade', 'Grid Stability And Storage', 'Land Use', 'Local Economy', 'Subsidy And Tariff Debate', 'Utility Bills']

Housing classifier results for housing_examples:

Example 1: 'Solar panels can help reduce electricity bills over time.'
Predictions (all label scores, highest first):
  Grid Stability And Storage: 0.431048
  Utility Bills: 0.283009
  Decommissioning And Waste: 0.195417
  Land Use: 0.085058
  Subsidy And Tariff Debate: 0.074950
  Local Economy: 0.063686
  Foreign Dependence And Trade: 0.032112
----------------------------------------
Example 2: 'Disposing of old solar panels is a growing environmental concern.'
Predictions (all label scores, highest first):
  Decommissioning And Waste: 0.611688
  Subsidy And Tariff Debate: 0.135908
  Foreign Dependence And Trade: 0.134344
  Local Economy: 0.117878
  Land Use: 0.109791
  Utility Bills: 0.065834
  Grid Stability And Storage: 0.045119
--

Fetching 7 files: 100%|██████████| 7/7 [00:09<00:00,  1.38s/it]


Model loaded successfully
   Labels: ['Animal Welfare', 'Environmental Impact', 'Health', 'Lab Grown And Alt Proteins', 'Psychology And Identity', 'Systemic Vs Individual Action', 'Taste And Convenience']

Food classifier results for food_examples:

Example 1: 'A plant-based diet is better for the environment.'
Predictions (all label scores, highest first):
  Environmental Impact: 0.982813
  Health: 0.117133
  Lab Grown And Alt Proteins: 0.097548
  Animal Welfare: 0.078281
  Psychology And Identity: 0.039711
  Systemic Vs Individual Action: 0.034358
  Taste And Convenience: 0.033903
----------------------------------------
Example 2: 'I can't give up cheese, even though I know veganism is healthier.'
Predictions (all label scores, highest first):
  Health: 0.603810
  Taste And Convenience: 0.293494
  Psychology And Identity: 0.179412
  Lab Grown And Alt Proteins: 0.099888
  Animal Welfare: 0.050795
  Environmental Impact: 0.036347
  Systemic Vs Individual Action: 0.034212
-------------

In [ ]:
import torch,sys,os,tempfile
import numpy as np
from transformers import DistilBertTokenizer
from huggingface_hub import snapshot_download

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def print_sorted_label_scores(label_scores):
    # Sort label_scores dict by score descending
    sorted_items = sorted(label_scores.items(), key=lambda x: x[1], reverse=True)
    for label, score in sorted_items:
        print(f"  {label}: {score:.6f}")

# Download and load model
model_link = "sanchow/veganism_and_vegetarianism-distilbert-classifier"
with tempfile.TemporaryDirectory() as temp_dir:
    snapshot_download(
        repo_id=model_link,
        local_dir=temp_dir,
        local_dir_use_symlinks=False
    )
    
    # Import the model class
    sys.path.insert(0, temp_dir)
    from model_class import MultilabelClassifier
    
    # Load tokenizer and model
    tokenizer = DistilBertTokenizer.from_pretrained(temp_dir)
    checkpoint = torch.load(os.path.join(temp_dir, 'model.pt'), map_location='cpu', weights_only=False)
    model = MultilabelClassifier(checkpoint['model_name'], len(checkpoint['label_names']))
    model.load_state_dict(checkpoint['model_state_dict'])
    model.to(device)
    model.eval()
    print("Model loaded successfully")
    print(f"   Labels: {checkpoint['label_names']}")

# Example usage with multiple examples
examples = [
    "You know much carbon a cow .",
    "Your second example text here.",
    "Your third example text here."
]

for i, test_text in enumerate(examples):
    inputs = tokenizer(
        test_text, 
        return_tensors="pt", 
        truncation=True, 
        max_length=512,
        padding=True
    ).to(device)
    with torch.no_grad():
        outputs = model(**inputs)
        predictions = outputs.cpu().numpy() if isinstance(outputs, (tuple, list)) else outputs.cpu().numpy()
    label_scores = {label: float(score) for label, score in zip(checkpoint['label_names'], predictions[0])}
    print(f"Example {i+1}: '{test_text}'")
    print("Predictions (all label scores, highest first):")
    print_sorted_label_scores(label_scores)
    print("-" * 40)


c:\Users\chowdhary\Desktop\Redit project 4.09.2024\sbert_env\Lib\site-packages\huggingface_hub\file_download.py:980: UserWarning: `local_dir_use_symlinks` parameter is deprecated and will be ignored. The process to download files to a local folder has been updated and do not rely on symlinks anymore. You only need to pass a destination folder as`local_dir`.
For more details, check out https://huggingface.co/docs/huggingface_hub/main/en/guides/download#download-files-to-local-folder.
  warnings.warn(
Fetching 7 files: 100%|██████████| 7/7 [00:03<00:00,  1.78it/s]


Model loaded successfully
   Labels: ['Animal Welfare', 'Environmental Impact', 'Health', 'Lab Grown And Alt Proteins', 'Psychology And Identity', 'Systemic Vs Individual Action', 'Taste And Convenience']
Example 1: 'Your first example text here.'
Predictions (all label scores, highest first):
  Environmental Impact: 0.322079
  Psychology And Identity: 0.194179
  Systemic Vs Individual Action: 0.175152
  Taste And Convenience: 0.114281
  Health: 0.112443
  Lab Grown And Alt Proteins: 0.085422
  Animal Welfare: 0.075806
----------------------------------------
Example 2: 'Your second example text here.'
Predictions (all label scores, highest first):
  Environmental Impact: 0.323291
  Psychology And Identity: 0.194841
  Systemic Vs Individual Action: 0.187242
  Health: 0.119370
  Taste And Convenience: 0.116698
  Lab Grown And Alt Proteins: 0.094659
  Animal Welfare: 0.074074
----------------------------------------
Example 3: 'Your third example text here.'
Predictions (all label scores

In [ ]:
import torch, sys, os, tempfile
from transformers import DistilBertTokenizer
from huggingface_hub import snapshot_download

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')

def print_sorted_label_scores(label_scores):
    # Sort label_scores dict by score descending
    sorted_items = sorted(label_scores.items(), key=lambda x: x[1], reverse=True)
    for label, score in sorted_items:
        print(f"  {label}: {score:.6f}")



MODEL: ELECTRIC VEHICLES SECTOR


Fetching 7 files: 100%|██████████| 7/7 [00:04<00:00,  1.59it/s]


Model loaded successfully
   Labels: ['Alternative Modes', 'Charging Infrastructure', 'Environmental Benefit', 'Grid Impact And Energy Mix', 'Mineral Supply Chain', 'Policy And Mandates', 'Purchase Price']

Electric Vehicles classifier results:

Example 1: 'Electric vehicles cost money dude.'
Predictions (all label scores, highest first):
  Charging Infrastructure: 0.377133
  Purchase Price: 0.266446
  Alternative Modes: 0.197600
  Policy And Mandates: 0.115848
  Mineral Supply Chain: 0.049927
  Environmental Benefit: 0.038453
  Grid Impact And Energy Mix: 0.012627
----------------------------------------


#PRO ANTI EXAMPLES

In [95]:
import pandas as pd

# Load the CSV file
df = pd.read_csv('paper4data/pro_anti_all_examples.csv')

# Calculate sectorwise accuracy
sector_accuracy = (
    df.groupby('sector')['is_correct']
    .mean()
    .reset_index()
    .rename(columns={'is_correct': 'accuracy'})
)

print(sector_accuracy)


      sector  accuracy
0       food  0.601093
1    housing  0.654822
2  transport  0.721649


# Loads vegans data

In [ ]:
import pandas as pd

# Define file paths
comments_path = r'H:\Redit project 4.09.2024\Reddit data\comments_reddit.csv'
submissions_path = r'H:\Redit project 4.09.2024\Reddit data\submissions_reddit.csv'

# Load CSVs
comments_df = pd.read_csv(comments_path, low_memory=False)
submissions_df = pd.read_csv(submissions_path, low_memory=False)

# Isolate rows where subreddit is 'vegan'
comments_vegan = comments_df[comments_df['subreddit'].str.lower() == 'vegan']
submissions_vegan = submissions_df[submissions_df['subreddit'].str.lower() == 'vegan']

In [ ]:
# Save the vegan comments and submissions DataFrames to CSV in the paper4data directory
output_dir = 'paper4data'
os.makedirs(output_dir, exist_ok=True)

comments_vegan.to_csv(os.path.join(output_dir, 'comments_vegan.csv'), index=False)
submissions_vegan.to_csv(os.path.join(output_dir, 'submissions_vegan.csv'), index=False)

In [ ]:
# Get all unique subreddits
unique_subreddits = comments_df['subreddit'].dropna().str.lower().unique()

# Find subreddits whose name contains 'meat' or 'veg' (case-insensitive)
matching_subreddits = [sub for sub in unique_subreddits if ('meat' in sub) or ('veg' in sub)]

# Filter comments where subreddit is in the matching set
comments_meat_veg = comments_df[comments_df['subreddit'].str.lower().isin(matching_subreddits)]
# Get counts of comments for only the matching subreddits and sort descending
matching_counts = comments_df[comments_df['subreddit'].str.lower().isin(matching_subreddits)]['subreddit'].str.lower().value_counts().sort_values(ascending=False)
matching_counts

In [ ]:
# Save comments and submissions DataFrames for the specified subreddits to CSV in the paper4data directory

output_dir = 'paper4data'
os.makedirs(output_dir, exist_ok=True)

# List of subreddits to save (all lowercase for matching)
target_subreddits = [
    'vegan', 'vegancirclejerk', 'vegetarian', 'debateavegan', 'veganfitness',
    'veganrecipes', 'shittyveganfoodporn', 'vegande', 'antivegan', 'veganfoodporn',
    'meat', 'exvegans', 'veganuk'
]

# Save comments and submissions for each subreddit
for subreddit in target_subreddits:
    # Comments
    comments_sub = comments_df[comments_df['subreddit'].str.lower() == subreddit]
    comments_sub.to_csv(os.path.join(output_dir, f'comments_{subreddit}.csv'), index=False)
    # Submissions
    submissions_sub = submissions_df[submissions_df['subreddit'].str.lower() == subreddit]
    submissions_sub.to_csv(os.path.join(output_dir, f'submissions_{subreddit}.csv'), index=False)

##  context S- BERT analysis

In [ ]:
import importlib
import paper4_Sbert_analysis
importlib.reload(paper4_Sbert_analysis)
from paper4_Sbert_analysis import analyze_keyword_contexts

# Call the main analysis function 
keyword_context, strong_keywords = analyze_keyword_contexts(
    keyword_comment_map,
    sector_keyword_strength
)

Processing keywords: 100%|██████████| 23/23 [00:22<00:00,  1.03keyword/s]


In [ ]:
all_results_df

[                                               comment  \
 0    &gt;It just makes 0 practical sense for averag...   
 1    &gt; Norway is currently at 95% of new cars so...   
 2    &gt;Why not???\n\nIn North America 120v 15a is...   
 3    It's definitely a concern.  But remember that ...   
 4    &gt;Because they don't have plans to make it a...   
 ..                                                 ...   
 295  I think sometimes the problem is just having s...   
 296  Impact of global warming r/nature r/environmen...   
 297  "A meat-eater’s diet requires 17 times more la...   
 298  ethical vegan as an individual - and don’t und...   
 299  A big topic. Climate activists are downstream,...   
 
                       topic  pro  anti  pro_and_anti  neither  
 0         Electric Vehicles  0.0   1.0           0.0      0.0  
 1         Electric Vehicles  0.0   0.0           0.0      1.0  
 2         Electric Vehicles  0.0   1.0           0.0      0.0  
 3         Electric Vehicles  

# Sbert contexts

In [ ]:
import os
import torch
from sentence_transformers import SentenceTransformer
from huggingface_hub import snapshot_download
import logging
from sklearn.metrics.pairwise import cosine_similarity
from tqdm import tqdm
import numpy as np
import pandas as pd

def setup_local_model(model_name='all-MiniLM-L6-v2'):
    """
    Download and setup the SBERT model locally.
    """
    cache_dir = os.path.join(os.getcwd(), 'models')
    os.makedirs(cache_dir, exist_ok=True)
    
    try:
        local_model_path = os.path.join(cache_dir, model_name)
        if not os.path.exists(local_model_path):
            print(f"Downloading model {model_name} to {local_model_path}...")
            snapshot_download(
                repo_id=f"sentence-transformers/{model_name}",
                local_dir=local_model_path,
                local_dir_use_symlinks=False
            )
        
        device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
        print(f"Device set to use {device}")
        
        model = SentenceTransformer(local_model_path, device=device)
        return model
    
    except Exception as e:
        logging.error(f"Error setting up local model: {e}")
        return None

def get_comment_label_scores(model, comments_by_sector, sector_labels):
    """
    Get similarity scores between comments and labels.
    Returns a dataframe with comments and scores for each label.
    """
    if model is None:
        raise ValueError("Model not initialized properly")
        
    all_results = []
    
    # Process each sector
    for sector, comments in comments_by_sector.items():
        if not comments or sector not in sector_labels:
            continue
            
        labels = sector_labels[sector]
        comments_list = list(comments)
        
        # Get embeddings
        label_embeddings = model.encode(labels)
        
        # Process comments in batches
        batch_size = 250
        for i in tqdm(range(0, len(comments_list), batch_size)):
            batch = comments_list[i:i + batch_size]
            comment_embeddings = model.encode(batch)
            similarities = cosine_similarity(comment_embeddings, label_embeddings)
            
            # Create records for each comment
            for comment, scores in zip(batch, similarities):
                result = {
                    'comment': comment,
                    **{label: score for label, score in zip(labels, scores)}
                }
                all_results.append(result)
                
    return pd.DataFrame(all_results)

# Calculate comments by sector
comments_by_sector = {}
for sector in ['transport', 'housing', 'food']:
    strong_comments = set()
    weak_comments = set()
    
    if 'sectors' in locals() or 'sectors' in globals():
        strong_key = f'{sector}_strong'
        if strong_key in sectors:
            for kw in sectors[strong_key]:
                if 'keyword_unique_comments' in locals() and kw in keyword_unique_comments:
                    strong_comments.update(keyword_unique_comments[kw])
                
        weak_key = f'{sector}_weak'
        if weak_key in sectors:
            for kw in sectors[weak_key]:
                if 'keyword_unique_comments' in locals() and kw in keyword_unique_comments:
                    weak_comments.update(keyword_unique_comments[kw])
    
    comments_by_sector[sector] = strong_comments.union(weak_comments)

# Define sector labels
sector_labels = {
    'transport': [
        'battery cost', 'purchase price', 'range anxiety', 
        'charging infrastructure', 'environmental benefit',
        'running costs', 'reliability', 'fun to drive'
    ],
    'housing': [
        'local economy', 'utility bills', 'visual impact',
        'land use', 'tax revenue', 'wildlife'
    ],
    'food': [
        'health', 'environmental impact', 'animal welfare',
        'cost', 'taste and convenience'
    ]
}

# Setup model and get results
model = setup_local_model()

# Sample 1000 comments from each sector
sampled_comments = {
    sector: list(comments)[:1000] if len(comments) > 1000 else list(comments)
    for sector, comments in comments_by_sector.items()
}

# Get dataframe with comments and scores
results_df = get_comment_label_scores(model, sampled_comments, sector_labels)


Device set to use cpu


  0%|          | 0/4 [00:00<?, ?it/s]

 25%|██▌       | 1/4 [00:04<00:12,  4.12s/it]


KeyboardInterrupt: 

""


In [ ]:
def plot_sector_results(results_df, sector_labels, threshold=0.3):
    """
    Create visualization of sector classification results using SBERT scores.
    
    Args:
        results_df: DataFrame with comment text and label scores
        sector_labels: Dict mapping sectors to label sets
        threshold: Score threshold for counting a label match (default 0.3)
    """
    FONT_SIZE = 17
    
    # Create visualization
    fig = plt.figure(figsize=(14, 24))
    gs = fig.add_gridspec(3, 1, height_ratios=[1, 1, 1], hspace=0.3)

    # Plot results for each sector
    for idx, (sector, labels) in enumerate(sector_labels.items()):
        ax = fig.add_subplot(gs[idx, 0])
        
        # Count comments above threshold for each label
        label_counts = {}
        for label in labels:
            matches = results_df[results_df[label] > threshold]
            label_counts[label] = len(matches)
            
            # Print 3 example comments for first label only
            if label == labels[0]:
                print(f"\nExample comments for {label} (score > {threshold}):")
                for _, row in matches.head(3).iterrows():
                    print(f"- {row['comment'][:200]}...")
        
        # Calculate percentages
        total = len(results_df)
        labels = list(labels)
        freqs = [label_counts[label] for label in labels]
        pcts = [count/total * 100 for count in freqs]
        
        # Create horizontal bar plot
        y_pos = np.arange(len(labels))
        color = '#43AA8B' if sector == 'transport' else '#F94144' if sector == 'food' else '#FFB703'
        ax.barh(y_pos, pcts, color=color, alpha=0.8, height=0.6)
        
        # Customize plot
        ax.set_yticks(y_pos)
        ax.set_yticklabels([label.lower() for label in labels], fontsize=FONT_SIZE*1.54)
        ax.set_xlabel('% of comments', fontsize=FONT_SIZE*1.54, labelpad=10)
        ax.tick_params(axis='x', labelsize=FONT_SIZE*1.54)
        ax.set_title(sector.capitalize(), fontsize=FONT_SIZE*1.8, pad=20)
        
        # Add percentage and count labels
        for i, (pct, count) in enumerate(zip(pcts, freqs)):
            ax.text(pct + 1, i, f'{pct:.1f}% (n={count})', va='center',
                   fontsize=FONT_SIZE*1.32, color='#444444')
        
        # Style adjustments
        ax.spines['top'].set_visible(False)
        ax.spines['right'].set_visible(False)
        ax.spines['left'].set_visible(False)
        ax.spines['bottom'].set_visible(False)
        ax.grid(False)
        ax.set_axisbelow(True)

    try:
        plt.savefig('paper4figs/paper4_fig2.pdf', bbox_inches='tight', dpi=300)
        plt.savefig('paper4figs/paper4_fig2.png', bbox_inches='tight', dpi=300)
    except PermissionError:
        print("Warning: Could not save figures due to permission error")
    plt.close()

# Plot results using the results_df
plot_sector_results(results_df, sector_labels)


Example comments for battery cost (score > 0.3):
- The small battery idea is a great one. Most US households have more than one car, so shouldn't one of these be a small, inexpensive EV? I'd love something like an $8,000 BYD with a 120-mile battery to...
- It really depends. If you use a rough estimate of 1 kWh/panel per day, and you want to charge your P100D Tesla (100  kWh battery) from 0 to 100%, you’d need 100 panels.\n\nThat is worst case, of cours...
- A battery like the one in that trailer very well might be able to do a DC quick charge at 50 kW.  An electric car like the one charging can drive several miles on a kilowatt-hour of energy.  So, 5 or ...

Example comments for local economy (score > 0.3):
- Yes but so what ? Taxing the people = less consumption = a nice blow to the economy. \n\nAnd all this for what ? Building useless solar panels and electric cars that would solve a problem that doesn’r...
- Upcoming community solar projects in Jersey help fight income disparities

# Regex based subsectors

In [ ]:
# Load regex patterns from JSON files
import json
from tqdm import tqdm

with open('paper4data/regex_improved.json', 'r') as f:
    regex_patterns = json.load(f)

transport_regex = regex_patterns['transport']
housing_regex = regex_patterns['housing'] 
food_regex = regex_patterns['food']

# Calculate comments by sector
comments_by_sector = {}
for sector in ['transport', 'housing', 'food']:
    strong_comments = set()
    weak_comments = set()
    
    if 'sectors' in locals() or 'sectors' in globals():
        strong_key = f'{sector}_strong'
        if strong_key in sectors:
            for kw in sectors[strong_key]:
                if 'keyword_unique_comments' in locals() and kw in keyword_unique_comments:
                    strong_comments.update(keyword_unique_comments[kw])
                
        weak_key = f'{sector}_weak'
        if weak_key in sectors:
            for kw in sectors[weak_key]:
                if 'keyword_unique_comments' in locals() and kw in keyword_unique_comments:
                    weak_comments.update(keyword_unique_comments[kw])
    
    comments_by_sector[sector] = strong_comments.union(weak_comments)

# Sample comments from each sector
sampled_comments = {
    sector: list(comments)[0:20000] if len(comments) > 1000 else list(comments)
    for sector, comments in comments_by_sector.items()
}

# Analyze comments for each sector with appropriate regex patterns
sector_matches = {}
regex_patterns = {
    'transport': transport_regex,
    'housing': housing_regex,
    'food': food_regex
}

for sector in ['transport', 'housing', 'food']:
    matches = {category: 0 for category in regex_patterns[sector].keys()}
    total_comments = len(sampled_comments[sector])
    
    print(f"\nProcessing {sector} comments:")
    for comment in tqdm(sampled_comments[sector], total=total_comments):
        for category, patterns in regex_patterns[sector].items():
            # Check if comment matches the regex patterns
            matched = False
            
            # Check must-have patterns if they exist
            if "must" in patterns:
                must_match = any(re.search(pat, comment, re.IGNORECASE) for pat in patterns["must"])
                if not must_match:
                    continue
                    
            # Check any patterns
            if "any" in patterns:
                any_match = any(re.search(pat, comment, re.IGNORECASE) for pat in patterns["any"])
                if not any_match:
                    continue
                    
            # Check exclude patterns if they exist
            if "exclude" in patterns:
                exclude_match = any(re.search(pat, comment, re.IGNORECASE) for pat in patterns["exclude"])
                if exclude_match:
                    continue
                    
            matches[category] += 1
            
    sector_matches[sector] = matches

# Print percentages for each sector and category
for sector in ['transport', 'housing', 'food']:
    total_comments = len(sampled_comments[sector])
    print(f"\nPercentage of {sector} comments matching each category:")
    for category, count in sector_matches[sector].items():
        percentage = (count / total_comments) * 100
        print(f"{category}: {percentage:.1f}% ({count} comments)")




Processing transport comments:


100%|██████████| 14924/14924 [00:07<00:00, 2016.54it/s]



Processing housing comments:


100%|██████████| 9388/9388 [00:03<00:00, 2857.76it/s]



Processing food comments:


100%|██████████| 8414/8414 [00:04<00:00, 1819.39it/s]



Percentage of transport comments matching each category:
Battery Cost: 1.0% (146 comments)
Purchase Price: 0.1% (14 comments)
Charging Infrastructure: 0.7% (101 comments)
Environmental Benefit: 1.9% (280 comments)
Running Costs: 0.1% (20 comments)
Reliability: 1.0% (147 comments)
Fun to Drive: 0.0% (0 comments)

Percentage of housing comments matching each category:
utility_bills: 4.8% (453 comments)
visual_impact: 0.0% (0 comments)
tax_revenue: 0.0% (0 comments)
wildlife: 0.3% (24 comments)

Percentage of food comments matching each category:
cost: 1.7% (141 comments)
health: 7.3% (612 comments)
environmental_impact: 13.6% (1147 comments)
animal_welfare: 2.6% (220 comments)
taste_convenience: 5.3% (450 comments)


=== EXAMPLE COMMENTS FROM TRANSPORT SECTOR ===

--- Battery Cost ---
Did you know Redwood Materials can recycle around 90% of the battery material, and then sell the components back to battery manufacturers, basically closing the loop, which could potentially help with the 

In [ ]:
 # Print 5 example comments for each sector and category for manual inspection
for sector in ['transport', 'housing', 'food']:
    print(f"\n\n=== EXAMPLE COMMENTS FROM {sector.upper()} SECTOR ===")

    # Get all comments for this sector
    sector_comments = sampled_comments[sector]
    sector_specific_regex = regex_patterns[sector]

    for category in sector_specific_regex.keys():
        print(f"\n--- {category} ---")
        examples = []
        count = 0
        
        # Find comments matching this category's patterns
        for comment in sector_comments:
            if count >= 5:
                break
                
            patterns = sector_specific_regex[category]
            matched = True
            
            # Check must-have patterns
            if "must" in patterns:
                must_match = False
                for pat in patterns["must"]:
                    for sentence in comment.split('.'):
                        if re.search(pat, sentence, re.IGNORECASE):
                            must_match = True
                            matched_sentence = sentence
                if not must_match:
                    continue
                    
            # Check any patterns  
            if "any" in patterns:
                any_match = False
                for pat in patterns["any"]:
                    for sentence in comment.split('.'):
                        if re.search(pat, sentence, re.IGNORECASE):
                            any_match = True
                            matched_sentence = sentence
                if not any_match:
                    continue
                    
            # Check exclude patterns
            if "exclude" in patterns:
                exclude_match = False
                for pat in patterns["exclude"]:
                    for sentence in comment.split('.'):
                        if re.search(pat, sentence, re.IGNORECASE):
                            exclude_match = True
                if exclude_match:
                    continue
                    
            examples.append(matched_sentence.strip())
            count += 1
            
        # Print examples found
        for i, example in enumerate(examples, 1):
            print('- ' + example)



=== EXAMPLE COMMENTS FROM TRANSPORT SECTOR ===

--- Battery Cost ---
- Did you know Redwood Materials can recycle around 90% of the battery material, and then sell the components back to battery manufacturers, basically closing the loop, which could potentially help with the EV battery shortage &amp; reduce the mining burden?\n\nThey are basically solving the problem of what to do with dead Li-ion batteries and giving them new life in brand new cells
- If you don't recycle them
- \n\n    Over the past five years alone, the Energiewende has cost Germany €32 billion ($36 billion) annually, and opposition to renewables is growing in the German countryside
- Yet, no one seems willing to try and recycle the supposedly eco-friendly vehicle
- \n\nThat was until high-profile LBC news radio personality Iain Dale bought an Audi e-tron (Prices start at £80,000-$100,000 after tax and before subsidies)

--- Purchase Price ---
- It’s literally under $20k with the tax credit
- \n\nAnd in the case o

# SBERT/ FaceBook BART

In [ ]:
# from paper4_BART_arguments import main_sbert
# from importlib import reload
# import paper4_BART_arguments
# paper4_BART_arguments = reload(paper4_BART_arguments)
# from paper4_BART_arguments import main_sbert
# # Sample comments from each sector
# sampled_comments = {
#     sector: list(comments)[500:600] if len(comments) > 350 else list(comments)
#     for sector, comments in comments_by_sector.items()
# }
# # Use BART instead
# results, results_df, sector_labels = main_sbert(
#     comments_by_sector=sampled_comments,
#     num_runs=1,
#     max_comments_per_sector=100,
#     num_workers=4,
#     batch_size=40,
#     threshold=0.5,
#     use_bart=False  # Switch to BART
# )

In [ ]:
# THRESHOLD = 0.7
# # Print overall statistics first
# print("=== Overall Statistics ===")
# for sector in sector_labels.keys():
#     sector_df = results_df[results_df['sector'] == sector].copy()
#     total_comments = len(sector_df)
#     print(f"\n{sector.upper()} SECTOR:")
#     print(f"Total comments: {total_comments}")
        
#     # Count unlabeled comments (including newly added ones)
#     no_labels = sector_df.copy()
#     for label in sector_labels[sector]:
#         col_name = f"{sector}_{label.lower().replace(' ', '_')}"
#         no_labels = no_labels[no_labels[col_name] < THRESHOLD]  # Using threshold
#     unlabeled_count = len(no_labels)
#     print(f"Unlabeled comments: {unlabeled_count}")
        
#     # Count comments per label
#     print("Comments per label:")
#     for label in sector_labels[sector]:
#         label_col = f"{sector}_{label.lower().replace(' ', '_')}"
#         labeled_count = len(sector_df[sector_df[label_col] >= THRESHOLD])
#         print(f"- {label}: {labeled_count}")
#     print("-" * 50)

# # Create single output file for all sectors
# with open(f'paper4data/gpt3_check_all_sectors_Sbert.txt', 'w', encoding='utf-8') as f:
#     # Add meta description at top
#     f.write("=" * 80 + "\n")
#     f.write("FILE CONTENTS DESCRIPTION:\n")
#     f.write("This file contains all comments from all sectors with their applied labels\n")
#     f.write("Format: [Label1, Label2, ...] Comment text\n")
#     f.write("Comments longer than 4000 characters are truncated with '...'\n")
#     f.write("=" * 80 + "\n\n")
        
#     # Process each sector
#     for sector in sector_labels.keys():
#         f.write(f"\n{sector.upper()} SECTOR COMMENTS:\n")
#         f.write("=" * 80 + "\n\n")
            
#         # Get comments in original order from sampled_comments
#         sector_comments = sampled_comments[sector]
        
#         # Process each comment in original order
#         for comment in sector_comments:
#             # Find matching row in results_df
#             row = results_df[(results_df['sector'] == sector) & (results_df['comment'] == comment)].iloc[0]
            
#             # Collect all labels that apply to this comment
#             applied_labels = []
#             for label in sector_labels[sector]:
#                 col_name = f"{sector}_{label.lower().replace(' ', '_')}"
#                 if row[col_name] >= THRESHOLD:
#                     applied_labels.append(label)
                
#             # Format comment text
#             comment_text = comment[:4000] + "..." if len(comment) > 4000 else comment
                
#             # Write comment with all its labels in one bracket
#             label_text = ", ".join(applied_labels) if applied_labels else "NO LABELS"
#             f.write(f"[{label_text}] {comment_text}\n\n")

# Actors

In [ ]:
# # Import spacy and download model if needed
# import spacy
# from tqdm import tqdm
# import time

# try:
#     nlp = spacy.load('en_core_web_sm')
# except:
#     import sys
#     import subprocess
#     subprocess.check_call([sys.executable, "-m", "spacy", "download", "en_core_web_sm"])
#     nlp = spacy.load('en_core_web_sm')

# # Function to extract nouns from text
# def extract_nouns(text):
#     if pd.isna(text):
#         return []
#     doc = nlp(text)
#     nouns = [token.text for token in doc if token.pos_ in ['NOUN', 'PROPN']]
#     return nouns

# # Add nouns column to all_results_df with progress bar
# print("Processing nouns for all_results_df...")
# start_time = time.time()
# tqdm.pandas(desc="Extracting nouns", unit="comments")
# all_results_df['nouns'] = all_results_df['comment_text'].progress_apply(extract_nouns)

# # First, identify comments in df_joint_nuclear that aren't in all_results_df
# missing_comments = df_joint_nuclear[~df_joint_nuclear['Document'].isin(all_results_df['comment_text'])]
# GPT_threshold=0

# # Create DataFrame with missing comments and 0 scores for all trials
# missing_df = pd.DataFrame({
#     'comment_text': missing_comments['Document'],
#     'trial_1': 0,
#     'trial_2': 0, 
#     'trial_3': 0,
#     'trial_4': 0,
#     'trial_5': 0,
#     'trial_6': 0,
#     'trial_7': 0,
#     'trial_8': 0,
#     'trial_9': 0,
#     'mean_score': 0
# })

# # Add nouns column to missing_df with progress bar
# print("\nProcessing nouns for missing comments...")
# tqdm.pandas(desc="Extracting nouns", unit="comments") 
# missing_df['nouns'] = missing_df['comment_text'].progress_apply(extract_nouns)

# # Concatenate with all_results_df
# all_results_df = pd.concat([all_results_df, missing_df], ignore_index=True)

# # Now create combined DataFrame with all metrics by merging on text content
# merged_df = pd.merge(
#     all_results_df,
#     df_joint_nuclear,
#     left_on='comment_text',
#     right_on='Document',
#     how='inner'
# )

# elapsed_time = time.time() - start_time
# print(f"\nTotal processing time: {elapsed_time:.2f} seconds")


In [ ]:
# !pip install spacy
# merged_df[['Document','nouns']]